In [30]:
# Kết nối Google Drive (nếu chạy trong Colab)
try:
    from google.colab import drive
    print("Kết nối Google Drive...")
    drive.mount('/content/drive')
except:
    print("Không thể kết nối Google Drive hoặc không chạy trong môi trường Colab.")

Kết nối Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **explore_data_detailed**

In [31]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import os
from scipy.fft import fft, fftfreq
import pandas as pd

# Đường dẫn đến file dữ liệu
data_path = '/content/drive/MyDrive/bidmc/bidmc-ppg-and-respiration-dataset-1.0.0/bidmc_data.mat'
figures_path = '/content/drive/MyDrive/bidmc/figures'
os.makedirs(figures_path, exist_ok=True)

# Tải dữ liệu
print("Đang tải dữ liệu từ file .mat...")
mat_data = sio.loadmat(data_path)
data = mat_data['data'][0]  # Lấy mảng chính chứa 53 bản ghi

# Khám phá cấu trúc dữ liệu
print(f"Số lượng bản ghi: {len(data)}")

# Khám phá cấu trúc chi tiết của bản ghi đầu tiên
first_record = data[0]
print("\nKhám phá cấu trúc chi tiết của bản ghi đầu tiên:")

# Kiểm tra cấu trúc của trường ppg
ppg_field = first_record['ppg'][0, 0]
print(f"Cấu trúc của trường ppg: {type(ppg_field)}")
if hasattr(ppg_field, 'dtype') and hasattr(ppg_field.dtype, 'names'):
    print(f"Các trường con của ppg: {ppg_field.dtype.names}")

# Kiểm tra cấu trúc của trường ref
ref_field = first_record['ref'][0, 0]
print(f"Cấu trúc của trường ref: {type(ref_field)}")
if hasattr(ref_field, 'dtype') and hasattr(ref_field.dtype, 'names'):
    print(f"Các trường con của ref: {ref_field.dtype.names}")

    # Kiểm tra cấu trúc của trường params trong ref
    if 'params' in ref_field.dtype.names:
        params_field = ref_field['params'][0, 0]
        print(f"Cấu trúc của trường params: {type(params_field)}")
        if hasattr(params_field, 'dtype') and hasattr(params_field.dtype, 'names'):
            print(f"Các trường con của params: {params_field.dtype.names}")

# Kiểm tra chi tiết hơn về cấu trúc dữ liệu
print("\nKiểm tra chi tiết hơn về cấu trúc dữ liệu:")
print(f"Kiểu dữ liệu của ppg.v: {type(first_record['ppg'][0, 0]['v'])}")
print(f"Kích thước của ppg.v: {first_record['ppg'][0, 0]['v'].shape}")

# Kiểm tra cấu trúc của HR và RR
print("\nKiểm tra cấu trúc của HR và RR:")
hr_field = first_record['ref'][0, 0]['params'][0, 0]['hr'][0]
print(f"Kiểu dữ liệu của hr: {type(hr_field)}")
print(f"Kích thước của hr: {hr_field.shape}")
print(f"Giá trị đầu tiên của hr: {hr_field[0] if hr_field.size > 0 else 'Không có dữ liệu'}")

rr_field = first_record['ref'][0, 0]['params'][0, 0]['rr'][0]
print(f"Kiểu dữ liệu của rr: {type(rr_field)}")
print(f"Kích thước của rr: {rr_field.shape}")
print(f"Giá trị đầu tiên của rr: {rr_field[0] if rr_field.size > 0 else 'Không có dữ liệu'}")

# Trích xuất và vẽ tín hiệu PPG từ bản ghi đầu tiên
try:
    # Truy cập trực tiếp vào dữ liệu PPG
    ppg_data = first_record['ppg'][0, 0]['v']
    if isinstance(ppg_data, np.ndarray):
        # Nếu là mảng numpy, lấy dữ liệu trực tiếp
        sample_ppg = ppg_data.flatten()
    else:
        # Nếu không phải mảng numpy, chuyển đổi thành mảng
        sample_ppg = np.array(ppg_data, dtype=float).flatten()

    # Lấy tần số lấy mẫu
    sample_fs_ppg = float(first_record['ppg'][0, 0]['fs'][0, 0])

    # Tương tự cho ECG và Resp
    ecg_data = first_record['ekg'][0, 0]['v']
    if isinstance(ecg_data, np.ndarray):
        sample_ecg = ecg_data.flatten()
    else:
        sample_ecg = np.array(ecg_data, dtype=float).flatten()

    sample_fs_ecg = float(first_record['ekg'][0, 0]['fs'][0, 0])

    resp_data = first_record['ref'][0, 0]['resp_sig'][0, 0]['imp'][0, 0]['v']
    if isinstance(resp_data, np.ndarray):
        sample_resp = resp_data.flatten()
    else:
        sample_resp = np.array(resp_data, dtype=float).flatten()

    sample_fs_resp = float(first_record['ref'][0, 0]['resp_sig'][0, 0]['imp'][0, 0]['fs'][0, 0])

    print(f"\nTần số lấy mẫu PPG: {sample_fs_ppg} Hz")
    print(f"Tần số lấy mẫu ECG: {sample_fs_ecg} Hz")
    print(f"Tần số lấy mẫu Resp: {sample_fs_resp} Hz")

    print(f"Độ dài tín hiệu PPG: {len(sample_ppg)} mẫu")
    print(f"Độ dài tín hiệu ECG: {len(sample_ecg)} mẫu")
    print(f"Độ dài tín hiệu Resp: {len(sample_resp)} mẫu")

    # Tính thời gian cho trục x
    time_ppg = np.arange(len(sample_ppg)) / sample_fs_ppg
    time_ecg = np.arange(len(sample_ecg)) / sample_fs_ecg
    time_resp = np.arange(len(sample_resp)) / sample_fs_resp

    # Vẽ tín hiệu mẫu
    plt.figure(figsize=(15, 10))

    plt.subplot(3, 1, 1)
    plt.plot(time_ecg[:1000], sample_ecg[:1000])
    plt.title('ECG Signal (First Record - First 1000 samples)')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')

    plt.subplot(3, 1, 2)
    plt.plot(time_ppg[:1000], sample_ppg[:1000])
    plt.title('PPG Signal (First Record - First 1000 samples)')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')

    plt.subplot(3, 1, 3)
    plt.plot(time_resp[:1000], sample_resp[:1000])
    plt.title('Respiratory Signal (First Record - First 1000 samples)')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')

    plt.tight_layout()
    plt.savefig(os.path.join(figures_path, 'sample_signals.png'))
    plt.close()

    # Phân tích phổ tần số của tín hiệu PPG
    def plot_fft(signal, fs, title, filename):
        N = len(signal)
        T = 1.0 / fs
        yf = fft(signal)
        xf = fftfreq(N, T)[:N//2]

        plt.figure(figsize=(10, 6))
        plt.plot(xf, 2.0/N * np.abs(yf[0:N//2]))
        plt.grid(True, alpha=0.3)
        plt.title(f'FFT of {title}')
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('Amplitude')
        plt.xlim(0, 5)  # Giới hạn tần số hiển thị đến 5Hz
        plt.savefig(os.path.join(figures_path, filename))
        plt.close()

    # Phân tích phổ tần số của tín hiệu PPG, ECG và Resp
    plot_fft(sample_ppg, sample_fs_ppg, 'PPG Signal', 'ppg_fft.png')
    plot_fft(sample_ecg, sample_fs_ecg, 'ECG Signal', 'ecg_fft.png')
    plot_fft(sample_resp, sample_fs_resp, 'Respiratory Signal', 'resp_fft.png')

    # Tạo báo cáo tóm tắt
    with open(os.path.join(figures_path, 'data_exploration_summary.txt'), 'w') as f:
        f.write("BÁO CÁO KHÁM PHÁ DỮ LIỆU BIDMC PPG AND RESPIRATION DATASET\n")
        f.write("==========================================================\n\n")

        f.write(f"Số lượng bản ghi: {len(data)}\n\n")

        f.write("Cấu trúc dữ liệu:\n")
        f.write("- Mỗi bản ghi chứa các trường: ppg, ekg, ref, fix\n")
        f.write("- Tín hiệu PPG và ECG được lưu trữ với giá trị (v) và tần số lấy mẫu (fs)\n")
        f.write("- Tín hiệu hô hấp được lưu trữ trong trường ref.resp_sig.imp\n")
        f.write("- Các thông số sinh lý (HR, RR, PR, SpO2) được lưu trữ trong trường ref.params\n\n")

        f.write(f"Tần số lấy mẫu PPG: {sample_fs_ppg} Hz\n")
        f.write(f"Tần số lấy mẫu ECG: {sample_fs_ecg} Hz\n")
        f.write(f"Tần số lấy mẫu Resp: {sample_fs_resp} Hz\n\n")

        f.write("Thách thức trong việc truy cập dữ liệu:\n")
        f.write("- Cấu trúc dữ liệu phức tạp với nhiều lớp lồng nhau\n")
        f.write("- Khó khăn trong việc chuyển đổi dữ liệu HR và RR sang định dạng float\n")
        f.write("- Cần phương pháp tiếp cận cẩn thận để trích xuất và xử lý dữ liệu\n\n")

        f.write("Các file đã tạo:\n")
        f.write("1. sample_signals.png - Biểu đồ mẫu của tín hiệu ECG, PPG và Respiratory\n")
        f.write("2. ppg_fft.png, ecg_fft.png, resp_fft.png - Phân tích phổ tần số của các tín hiệu\n")

    print("\nPhân tích dữ liệu hoàn tất. Các biểu đồ và báo cáo đã được lưu vào thư mục figures.")

except Exception as e:
    print(f"Lỗi khi vẽ tín hiệu mẫu: {e}")

# Kiểm tra trực tiếp một số bản ghi để tìm hiểu cấu trúc HR và RR
print("\nKiểm tra trực tiếp HR và RR từ một số bản ghi:")
for i in range(min(5, len(data))):
    try:
        record = data[i]
        params = record['ref'][0, 0]['params'][0, 0]

        hr_data = params['hr'][0]
        rr_data = params['rr'][0]

        print(f"\nBản ghi {i}:")
        print(f"Kiểu dữ liệu HR: {type(hr_data)}")
        if hasattr(hr_data, 'dtype'):
            print(f"Dtype của HR: {hr_data.dtype}")
        if hasattr(hr_data, 'shape'):
            print(f"Kích thước của HR: {hr_data.shape}")

        print(f"Kiểu dữ liệu RR: {type(rr_data)}")
        if hasattr(rr_data, 'dtype'):
            print(f"Dtype của RR: {rr_data.dtype}")
        if hasattr(rr_data, 'shape'):
            print(f"Kích thước của RR: {rr_data.shape}")

        # Thử in ra một số giá trị đầu tiên
        if hasattr(hr_data, 'size') and hr_data.size > 0:
            if hasattr(hr_data, 'dtype') and hr_data.dtype.names is not None and 'v' in hr_data.dtype.names:
                print(f"Giá trị HR đầu tiên (trường v): {hr_data['v'][0] if hr_data['v'].size > 0 else 'Không có dữ liệu'}")
            else:
                print(f"Giá trị HR đầu tiên: {hr_data[0]}")

        if hasattr(rr_data, 'size') and rr_data.size > 0:
            if hasattr(rr_data, 'dtype') and rr_data.dtype.names is not None and 'v' in rr_data.dtype.names:
                print(f"Giá trị RR đầu tiên (trường v): {rr_data['v'][0] if rr_data['v'].size > 0 else 'Không có dữ liệu'}")
            else:
                print(f"Giá trị RR đầu tiên: {rr_data[0]}")
    except Exception as e:
        print(f"Lỗi khi kiểm tra bản ghi {i}: {e}")


Streaming output truncated to the last 5000 lines.
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [20],
       [20],
       [20],
       [20],
       [20],
       [20],
       [20],
       [20],
       [20],
       [20],
       [20],
       [20],
       [20],
       [20],
       [20],
       [20],
       [20],
       [20],
       [20],
       [20],
       [20],
       [20],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],


# **check_data_structure**

In [32]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import os
from scipy.fft import fft, fftfreq
import pandas as pd

# Đường dẫn đến file dữ liệu
data_path = '/content/drive/MyDrive/bidmc/bidmc-ppg-and-respiration-dataset-1.0.0/bidmc_data.mat'
figures_path = '/content/drive/MyDrive/bidmc/figures'

# Tải dữ liệu
print("Đang tải dữ liệu từ file .mat...")
mat_data = sio.loadmat(data_path)

# Kiểm tra cấu trúc dữ liệu
print("Các khóa trong file .mat:")
for key in mat_data.keys():
    print(f"- {key}")

# Kiểm tra cấu trúc chi tiết
if 'data' in mat_data:
    data = mat_data['data']
    print(f"\nKiểu dữ liệu của 'data': {type(data)}")
    print(f"Kích thước của 'data': {data.shape}")

    # Kiểm tra cấu trúc của phần tử đầu tiên nếu data là mảng
    if isinstance(data, np.ndarray) and data.size > 0:
        first_record = data[0]
        if hasattr(first_record, 'dtype') and hasattr(first_record.dtype, 'names'):
            print("\nCác trường trong bản ghi đầu tiên:")
            for field in first_record.dtype.names:
                print(f"- {field}")

                # Kiểm tra cấu trúc chi tiết của từng trường
                field_data = first_record[field]
                print(f"  Kiểu: {type(field_data)}, Kích thước: {field_data.shape}")

                # Nếu là mảng có cấu trúc, hiển thị thêm thông tin
                if hasattr(field_data, 'dtype') and hasattr(field_data.dtype, 'names'):
                    print(f"  Các trường con: {field_data.dtype.names}")
        else:
            print(f"\nPhần tử đầu tiên không có cấu trúc trường, kiểu: {type(first_record)}")
    else:
        print("\nKhông thể truy cập phần tử đầu tiên của 'data'")
else:
    print("\nKhông tìm thấy khóa 'data' trong file .mat")
    print("Đang kiểm tra cấu trúc dữ liệu thay thế...")

    # Tìm khóa có thể chứa dữ liệu chính
    potential_data_keys = [k for k in mat_data.keys() if not k.startswith('__')]
    for key in potential_data_keys:
        print(f"\nKiểm tra khóa: {key}")
        data_item = mat_data[key]
        print(f"Kiểu: {type(data_item)}, ", end="")

        if isinstance(data_item, np.ndarray):
            print(f"Kích thước: {data_item.shape}")

            # Kiểm tra nếu là mảng có cấu trúc
            if data_item.dtype.names is not None:
                print(f"Các trường: {data_item.dtype.names}")

            # Kiểm tra phần tử đầu tiên nếu là mảng nhiều chiều
            if data_item.size > 0:
                first_item = data_item[0]
                print(f"Kiểu phần tử đầu tiên: {type(first_item)}")

                # Nếu phần tử đầu tiên là mảng có cấu trúc
                if hasattr(first_item, 'dtype') and hasattr(first_item.dtype, 'names'):
                    print(f"Các trường trong phần tử đầu tiên: {first_item.dtype.names}")
        else:
            print(f"Không phải mảng numpy")

print("\nĐang lưu thông tin cấu trúc dữ liệu vào file...")
with open(os.path.join(figures_path, 'data_structure.txt'), 'w') as f:
    f.write("CẤU TRÚC DỮ LIỆU BIDMC PPG AND RESPIRATION DATASET\n")
    f.write("=================================================\n\n")

    f.write("Các khóa trong file .mat:\n")
    for key in mat_data.keys():
        f.write(f"- {key}\n")

    f.write("\nThông tin chi tiết về cấu trúc dữ liệu được lưu trong file này.")

print("\nĐã lưu thông tin cấu trúc dữ liệu. Vui lòng kiểm tra file data_structure.txt")


Đang tải dữ liệu từ file .mat...
Các khóa trong file .mat:
- __header__
- __version__
- __globals__
- data

Kiểu dữ liệu của 'data': <class 'numpy.ndarray'>
Kích thước của 'data': (1, 53)

Các trường trong bản ghi đầu tiên:
- fix
  Kiểu: <class 'numpy.ndarray'>, Kích thước: (53,)
  Các trường con: None
- ppg
  Kiểu: <class 'numpy.ndarray'>, Kích thước: (53,)
  Các trường con: None
- ekg
  Kiểu: <class 'numpy.ndarray'>, Kích thước: (53,)
  Các trường con: None
- ref
  Kiểu: <class 'numpy.ndarray'>, Kích thước: (53,)
  Các trường con: None

Đang lưu thông tin cấu trúc dữ liệu vào file...

Đã lưu thông tin cấu trúc dữ liệu. Vui lòng kiểm tra file data_structure.txt


# **preprocess_data_fixed**

In [33]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import os
from scipy.signal import butter, filtfilt, resample
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import pandas as pd
from sklearn.model_selection import train_test_split

# Đường dẫn đến file dữ liệu
data_path = '/content/drive/MyDrive/bidmc/bidmc-ppg-and-respiration-dataset-1.0.0/bidmc_data.mat'
processed_data_path = '/content/drive/MyDrive/bidmc/processed'
figures_path = '/content/drive/MyDrive/bidmc/figures'

# Tạo thư mục nếu chưa tồn tại
os.makedirs(processed_data_path, exist_ok=True)
os.makedirs(figures_path, exist_ok=True)

# Tải dữ liệu
print("Đang tải dữ liệu từ file .mat...")
mat_data = sio.loadmat(data_path)
data = mat_data['data'][0]  # Lấy mảng chính chứa 53 bản ghi

print(f"Số lượng bản ghi: {len(data)}")

# Hàm lọc nhiễu cho tín hiệu PPG
def butter_bandpass_filter(data, lowcut, highcut, fs, order=2):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    return y

# Hàm chuẩn hóa tín hiệu (PPG) từ [min, max] thành [0, 1]
def normalize_signal(signal, method='minmax'):
    if method == 'minmax':
        scaler = MinMaxScaler(feature_range=(0, 1))  # Thay đổi từ (-1, 1) thành (0, 1)
        signal_reshaped = signal.reshape(-1, 1)
        normalized = scaler.fit_transform(signal_reshaped).flatten()
    elif method == 'standard':
        scaler = StandardScaler()
        signal_reshaped = signal.reshape(-1, 1)
        normalized = scaler.fit_transform(signal_reshaped).flatten()
    elif method == 'simple':
        normalized = (signal - np.mean(signal)) / np.std(signal)
    else:
        raise ValueError("Phương pháp chuẩn hóa không hợp lệ")
    return normalized

# Hàm chia tín hiệu thành các đoạn có độ dài cố định
def segment_signal(signal, segment_length, overlap=0):
    step = int(segment_length * (1 - overlap))
    segments = []
    for i in range(0, len(signal) - segment_length + 1, step):
        segments.append(signal[i:i + segment_length])
    return np.array(segments)

# Hàm trích xuất đặc trưng HR và BR từ tín hiệu (không chuẩn hóa)
def extract_hr_br_features(hr_values, rr_values):
    # Lấy giá trị trung bình của HR và BR, không chuẩn hóa
    hr_mean = np.mean(hr_values)
    rr_mean = np.mean(rr_values)
    return hr_mean, rr_mean

# Danh sách để lưu trữ dữ liệu đã tiền xử lý
ppg_segments = []
hr_features = []
rr_features = []

# Tham số tiền xử lý
fs = 125  # Tần số lấy mẫu (Hz)
segment_length = 8 * fs  # Độ dài đoạn tín hiệu (8 giây)
overlap = 0.5  # Độ chồng lấp giữa các đoạn (50%)
lowcut = 0.1  # Tần số cắt dưới cho bộ lọc (Hz)
highcut = 8.0  # Tần số cắt trên cho bộ lọc (Hz)

# Tiền xử lý dữ liệu từ mỗi bản ghi
valid_records = 0
for i in range(len(data)):
    try:
        record = data[i]

        # Trích xuất tín hiệu PPG
        ppg_data = record['ppg'][0, 0]['v']
        if isinstance(ppg_data, np.ndarray):
            ppg_signal = ppg_data.flatten()
        else:
            ppg_signal = np.array(ppg_data, dtype=float).flatten()

        # Trích xuất HR và RR - Truy cập trực tiếp vào giá trị số trong mảng
        try:
            # Trích xuất HR từ mảng lồng nhau
            hr_data = record['ref'][0, 0]['params'][0, 0]['hr'][0]
            if hasattr(hr_data, 'dtype') and hr_data.dtype.names is not None and 'v' in hr_data.dtype.names:
                hr_values_raw = hr_data['v']
            else:
                hr_values_raw = hr_data

            # Trích xuất RR từ mảng lồng nhau
            rr_data = record['ref'][0, 0]['params'][0, 0]['rr'][0]
            if hasattr(rr_data, 'dtype') and rr_data.dtype.names is not None and 'v' in rr_data.dtype.names:
                rr_values_raw = rr_data['v']
            else:
                rr_values_raw = rr_data

            # Trích xuất giá trị số từ mảng lồng nhau
            hr_values = []
            for item in hr_values_raw:
                if isinstance(item, np.ndarray) and item.size > 0:
                    hr_values.append(float(item[0]))
                elif np.isscalar(item):
                    hr_values.append(float(item))

            rr_values = []
            for item in rr_values_raw:
                if isinstance(item, np.ndarray) and item.size > 0:
                    rr_values.append(float(item[0]))
                elif np.isscalar(item):
                    rr_values.append(float(item))

            # Chuyển đổi sang mảng numpy
            hr_values = np.array(hr_values)
            rr_values = np.array(rr_values)

            # Kiểm tra xem có đủ dữ liệu không
            if len(hr_values) == 0 or len(rr_values) == 0:
                print(f"Không đủ dữ liệu HR/RR cho bản ghi {i}, bỏ qua bản ghi này")
                continue

        except Exception as e:
            print(f"Lỗi khi trích xuất HR/RR của bản ghi {i}: {e}, bỏ qua bản ghi này")
            continue

        # Lọc nhiễu tín hiệu PPG
        ppg_filtered = butter_bandpass_filter(ppg_signal, lowcut, highcut, fs)

        # Chuẩn hóa tín hiệu PPG (đã chỉnh thành [0, 1])
        ppg_normalized = normalize_signal(ppg_filtered, method='minmax')

        # Chia tín hiệu thành các đoạn
        segments = segment_signal(ppg_normalized, segment_length, overlap)

        # Trích xuất đặc trưng HR và BR cho mỗi đoạn (không chuẩn hóa)
        for segment in segments:
            hr_feature, rr_feature = extract_hr_br_features(hr_values, rr_values)

            # Thêm vào danh sách
            ppg_segments.append(segment)
            hr_features.append(hr_feature)
            rr_features.append(rr_feature)

        valid_records += 1
        print(f"Đã xử lý bản ghi {i}, số đoạn tín hiệu: {len(segments)}")

    except Exception as e:
        print(f"Lỗi khi xử lý bản ghi {i}: {e}")

print(f"\nĐã xử lý thành công {valid_records}/{len(data)} bản ghi")
print(f"Tổng số đoạn tín hiệu: {len(ppg_segments)}")

# Kiểm tra xem có đủ dữ liệu không
if len(ppg_segments) == 0:
    print("Không có đủ dữ liệu để tiếp tục. Sử dụng dữ liệu giả lập để minh họa.")

    # Tạo dữ liệu giả lập để minh họa
    num_samples = 100
    ppg_segments = np.random.rand(num_samples, segment_length)  # Giả lập tín hiệu PPG từ [0, 1]
    hr_features = np.random.uniform(60, 120, num_samples)  # HR từ 60-120 bpm (không chuẩn hóa)
    rr_features = np.random.uniform(6, 18, num_samples)  # RR từ 6-18 breaths/min (không chuẩn hóa)

    print(f"Đã tạo {num_samples} mẫu dữ liệu giả lập.")

# Chuyển đổi danh sách thành mảng numpy
ppg_segments = np.array(ppg_segments)
hr_features = np.array(hr_features)
rr_features = np.array(rr_features)

# Chia dữ liệu thành tập huấn luyện và tập kiểm thử
X_train, X_test, hr_train, hr_test, rr_train, rr_test = train_test_split(
    ppg_segments, hr_features, rr_features, test_size=0.2, random_state=42
)

# Lưu dữ liệu đã tiền xử lý
np.save(os.path.join(processed_data_path, 'ppg_train.npy'), X_train)
np.save(os.path.join(processed_data_path, 'ppg_test.npy'), X_test)
np.save(os.path.join(processed_data_path, 'hr_train.npy'), hr_train)
np.save(os.path.join(processed_data_path, 'hr_test.npy'), hr_test)
np.save(os.path.join(processed_data_path, 'rr_train.npy'), rr_train)
np.save(os.path.join(processed_data_path, 'rr_test.npy'), rr_test)

# Lưu thông tin về dữ liệu đã tiền xử lý
with open(os.path.join(processed_data_path, 'preprocessing_info.txt'), 'w') as f:
    f.write("THÔNG TIN TIỀN XỬ LÝ DỮ LIỆU\n")
    f.write("============================\n\n")

    f.write(f"Số lượng bản ghi đã xử lý: {valid_records}/{len(data)}\n")
    f.write(f"Tổng số đoạn tín hiệu: {len(ppg_segments)}\n\n")

    f.write("Tham số tiền xử lý:\n")
    f.write(f"- Tần số lấy mẫu: {fs} Hz\n")
    f.write(f"- Độ dài đoạn tín hiệu: {segment_length} mẫu ({segment_length/fs} giây)\n")
    f.write(f"- Độ chồng lấp: {overlap*100}%\n")
    f.write(f"- Tần số cắt dưới: {lowcut} Hz\n")
    f.write(f"- Tần số cắt trên: {highcut} Hz\n\n")

    f.write("Kích thước dữ liệu:\n")
    f.write(f"- Tập huấn luyện: {X_train.shape[0]} mẫu\n")
    f.write(f"- Tập kiểm thử: {X_test.shape[0]} mẫu\n\n")

    f.write("Thống kê HR (không chuẩn hóa):\n")
    f.write(f"- Min: {np.min(hr_features):.4f}, Max: {np.max(hr_features):.4f}\n")
    f.write(f"- Mean: {np.mean(hr_features):.4f}, Std: {np.std(hr_features):.4f}\n\n")

    f.write("Thống kê RR (không chuẩn hóa):\n")
    f.write(f"- Min: {np.min(rr_features):.4f}, Max: {np.max(rr_features):.4f}\n")
    f.write(f"- Mean: {np.mean(rr_features):.4f}, Std: {np.std(rr_features):.4f}\n")

    if valid_records == 0:
        f.write("\nLưu ý: Dữ liệu được sử dụng là dữ liệu giả lập do không thể trích xuất được dữ liệu thực từ bộ dữ liệu BIDMC.\n")

# Vẽ biểu đồ phân phối HR và RR (không chuẩn hóa)
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(hr_features, bins=30, alpha=0.7, color='blue')
plt.axvline(np.mean(hr_features), color='red', linestyle='dashed', linewidth=1)
plt.title('Heart Rate Distribution (bpm)')
plt.xlabel('Heart Rate (bpm)')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.hist(rr_features, bins=30, alpha=0.7, color='green')
plt.axvline(np.mean(rr_features), color='red', linestyle='dashed', linewidth=1)
plt.title('Respiratory Rate Distribution (breaths/min)')
plt.xlabel('Respiratory Rate (breaths/min)')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(figures_path, 'hr_rr_distribution.png'))
plt.close()

# Vẽ một số đoạn tín hiệu PPG đã tiền xử lý
plt.figure(figsize=(15, 10))

for i in range(min(5, len(X_train))):
    plt.subplot(5, 1, i+1)
    plt.plot(X_train[i])
    plt.title(f'Preprocessed PPG Segment {i+1}')
    plt.xlabel('Sample')
    plt.ylabel('Amplitude (Normalized [0, 1])')
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(figures_path, 'preprocessed_ppg_segments.png'))
plt.close()

print("\nTiền xử lý dữ liệu hoàn tất. Dữ liệu đã được lưu vào thư mục processed.")

Đang tải dữ liệu từ file .mat...
Số lượng bản ghi: 53
Đã xử lý bản ghi 0, số đoạn tín hiệu: 119
Đã xử lý bản ghi 1, số đoạn tín hiệu: 119
Đã xử lý bản ghi 2, số đoạn tín hiệu: 119
Đã xử lý bản ghi 3, số đoạn tín hiệu: 119
Đã xử lý bản ghi 4, số đoạn tín hiệu: 119
Đã xử lý bản ghi 5, số đoạn tín hiệu: 119
Đã xử lý bản ghi 6, số đoạn tín hiệu: 119
Đã xử lý bản ghi 7, số đoạn tín hiệu: 119
Đã xử lý bản ghi 8, số đoạn tín hiệu: 119
Đã xử lý bản ghi 9, số đoạn tín hiệu: 119
Đã xử lý bản ghi 10, số đoạn tín hiệu: 119
Đã xử lý bản ghi 11, số đoạn tín hiệu: 119
Đã xử lý bản ghi 12, số đoạn tín hiệu: 119
Đã xử lý bản ghi 13, số đoạn tín hiệu: 119
Đã xử lý bản ghi 14, số đoạn tín hiệu: 119
Đã xử lý bản ghi 15, số đoạn tín hiệu: 119
Đã xử lý bản ghi 16, số đoạn tín hiệu: 119
Đã xử lý bản ghi 17, số đoạn tín hiệu: 119
Đã xử lý bản ghi 18, số đoạn tín hiệu: 119
Đã xử lý bản ghi 19, số đoạn tín hiệu: 119
Đã xử lý bản ghi 20, số đoạn tín hiệu: 119
Đã xử lý bản ghi 21, số đoạn tín hiệu: 119


<ipython-input-33-08161e0b40d1>:111: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  hr_values.append(float(item[0]))
<ipython-input-33-08161e0b40d1>:118: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  rr_values.append(float(item[0]))
<ipython-input-33-08161e0b40d1>:111: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  hr_values.append(float(item[0]))
<ipython-input-33-08161e0b40d1>:118: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, 

Đã xử lý bản ghi 22, số đoạn tín hiệu: 119
Đã xử lý bản ghi 23, số đoạn tín hiệu: 119
Đã xử lý bản ghi 24, số đoạn tín hiệu: 119
Đã xử lý bản ghi 25, số đoạn tín hiệu: 119
Đã xử lý bản ghi 26, số đoạn tín hiệu: 119
Đã xử lý bản ghi 27, số đoạn tín hiệu: 119
Đã xử lý bản ghi 28, số đoạn tín hiệu: 119
Đã xử lý bản ghi 29, số đoạn tín hiệu: 119
Đã xử lý bản ghi 30, số đoạn tín hiệu: 119
Đã xử lý bản ghi 31, số đoạn tín hiệu: 119
Đã xử lý bản ghi 32, số đoạn tín hiệu: 119
Đã xử lý bản ghi 33, số đoạn tín hiệu: 119
Đã xử lý bản ghi 34, số đoạn tín hiệu: 119
Đã xử lý bản ghi 35, số đoạn tín hiệu: 119
Đã xử lý bản ghi 36, số đoạn tín hiệu: 119
Đã xử lý bản ghi 37, số đoạn tín hiệu: 119
Đã xử lý bản ghi 38, số đoạn tín hiệu: 119
Đã xử lý bản ghi 39, số đoạn tín hiệu: 119
Đã xử lý bản ghi 40, số đoạn tín hiệu: 119
Đã xử lý bản ghi 41, số đoạn tín hiệu: 119
Đã xử lý bản ghi 42, số đoạn tín hiệu: 119
Đã xử lý bản ghi 43, số đoạn tín hiệu: 119
Đã xử lý bản ghi 44, số đoạn tín hiệu: 119
Đã xử lý bả

<ipython-input-33-08161e0b40d1>:111: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  hr_values.append(float(item[0]))
<ipython-input-33-08161e0b40d1>:118: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  rr_values.append(float(item[0]))
<ipython-input-33-08161e0b40d1>:111: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  hr_values.append(float(item[0]))
<ipython-input-33-08161e0b40d1>:118: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, 

Đã xử lý bản ghi 48, số đoạn tín hiệu: 119
Đã xử lý bản ghi 49, số đoạn tín hiệu: 119
Đã xử lý bản ghi 50, số đoạn tín hiệu: 119
Đã xử lý bản ghi 51, số đoạn tín hiệu: 119
Đã xử lý bản ghi 52, số đoạn tín hiệu: 119

Đã xử lý thành công 53/53 bản ghi
Tổng số đoạn tín hiệu: 6307

Tiền xử lý dữ liệu hoàn tất. Dữ liệu đã được lưu vào thư mục processed.


# **cvae_model**

In [34]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam
import datetime

# Đường dẫn đến dữ liệu đã tiền xử lý
processed_data_path = '/content/drive/MyDrive/bidmc/processed'
model_path = '/content/drive/MyDrive/bidmc/models'
figures_path = '/content/drive/MyDrive/bidmc/figures'

# Tạo thư mục nếu chưa tồn tại
os.makedirs(model_path, exist_ok=True)
os.makedirs(figures_path, exist_ok=True)

# Tải dữ liệu đã tiền xử lý
print("Đang tải dữ liệu đã tiền xử lý...")
X_train = np.load(os.path.join(processed_data_path, 'ppg_train.npy'))
X_test = np.load(os.path.join(processed_data_path, 'ppg_test.npy'))
hr_train = np.load(os.path.join(processed_data_path, 'hr_train.npy'))
hr_test = np.load(os.path.join(processed_data_path, 'hr_test.npy'))
rr_train = np.load(os.path.join(processed_data_path, 'rr_train.npy'))
rr_test = np.load(os.path.join(processed_data_path, 'rr_test.npy'))

print(f"Kích thước dữ liệu huấn luyện: {X_train.shape}")
print(f"Kích thước dữ liệu kiểm thử: {X_test.shape}")

# Tham số mô hình
input_dim = X_train.shape[1]  # Độ dài đoạn tín hiệu PPG
condition_dim = 2  # HR và RR
latent_dim = 64  # Kích thước không gian tiềm ẩn
hidden_units = [256, 128, 64]  # Số đơn vị ẩn trong các lớp
batch_size = 64
epochs = 150
learning_rate = 0.0001

# Định nghĩa lớp Sampling để lấy mẫu từ không gian tiềm ẩn
class Sampling(layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

# Xây dựng Encoder
def build_encoder(input_dim, condition_dim, latent_dim, hidden_units):
    # Đầu vào tín hiệu PPG
    encoder_inputs = layers.Input(shape=(input_dim,), name='encoder_input')

    # Đầu vào điều kiện (HR và RR)
    condition_inputs = layers.Input(shape=(condition_dim,), name='condition_input')

    # Kết hợp đầu vào tín hiệu và điều kiện
    x = layers.Concatenate()([encoder_inputs, condition_inputs])

    # Các lớp ẩn
    for units in hidden_units:
        x = layers.Dense(units, activation='relu')(x)

    # Lớp đầu ra
    z_mean = layers.Dense(latent_dim, name='z_mean')(x)
    z_log_var = layers.Dense(latent_dim, name='z_log_var')(x)

    # Lấy mẫu từ không gian tiềm ẩn
    z = Sampling()([z_mean, z_log_var])

    # Định nghĩa mô hình
    encoder = Model([encoder_inputs, condition_inputs], [z_mean, z_log_var, z], name='encoder')

    return encoder

# Xây dựng Decoder
def build_decoder(latent_dim, condition_dim, input_dim, hidden_units):
    # Đầu vào từ không gian tiềm ẩn
    latent_inputs = layers.Input(shape=(latent_dim,), name='latent_input')

    # Đầu vào điều kiện (HR và RR)
    condition_inputs = layers.Input(shape=(condition_dim,), name='condition_input')

    # Kết hợp đầu vào từ không gian tiềm ẩn và điều kiện
    x = layers.Concatenate()([latent_inputs, condition_inputs])

    # Các lớp ẩn
    for units in reversed(hidden_units):
        x = layers.Dense(units, activation='relu')(x)

    # Lớp đầu ra
    decoder_outputs = layers.Dense(input_dim, activation='tanh')(x)

    # Định nghĩa mô hình
    decoder = Model([latent_inputs, condition_inputs], decoder_outputs, name='decoder')

    return decoder

# Xây dựng mô hình CVAE
class CVAE(Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(CVAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = tf.keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = tf.keras.metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker = tf.keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        x, condition = data
        print("x shape:", x.shape)
        print("condition shape:", condition.shape)

        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encode([x, condition])
            reconstruction = self.decode([z, condition])
            print("reconstruction shape:", reconstruction.shape)

            mse_result = tf.keras.losses.mse(x, reconstruction)
            print("mse result shape:", mse_result.shape)

            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(mse_result, axis=1)
            )
            kl_loss = -0.5 * tf.reduce_mean(
                tf.reduce_sum(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=1)
            )
            total_loss = reconstruction_loss + kl_loss

        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))

        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)

        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

    def test_step(self, data):
        if isinstance(data, tuple):
            x_condition = data[0]
            if isinstance(x_condition, list) and len(x_condition) == 2:
                x, condition = x_condition
            else:
                raise ValueError("Input data format is incorrect. Expected a list with [x, condition]")
        else:
            raise ValueError("Input data format is incorrect. Expected a tuple with ([x, condition], None)")

        # Encoder
        z_mean, z_log_var, z = self.encoder([x, condition])

        # Decoder
        reconstruction = self.decoder([z, condition])

        # Tính toán loss
        reconstruction_loss = tf.reduce_mean(
            tf.reduce_sum(
                tf.keras.losses.mse(x, reconstruction), axis=1
            )
        )

        kl_loss = -0.5 * tf.reduce_mean(
            tf.reduce_sum(
                1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=1
            )
        )

        total_loss = reconstruction_loss + kl_loss

        # Cập nhật metrics
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)

        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

    def call(self, inputs):
        x, condition = inputs
        z_mean, z_log_var, z = self.encoder([x, condition])
        reconstruction = self.decoder([z, condition])
        return reconstruction

    def generate(self, condition, z=None):
        if z is None:
            # Tạo vector ngẫu nhiên từ không gian tiềm ẩn
            z = tf.random.normal(shape=(condition.shape[0], latent_dim))

        # Tạo tín hiệu PPG từ vector z và điều kiện
        return self.decoder([z, condition])

# Xây dựng mô hình
print("Đang xây dựng mô hình CVAE...")
encoder = build_encoder(input_dim, condition_dim, latent_dim, hidden_units)
decoder = build_decoder(latent_dim, condition_dim, input_dim, hidden_units)
cvae = CVAE(encoder, decoder)

# Biên dịch mô hình
cvae.compile(optimizer=Adam(learning_rate=learning_rate))

# Tóm tắt mô hình
print("Tóm tắt mô hình Encoder:")
encoder.summary()
print("\nTóm tắt mô hình Decoder:")
decoder.summary()

# Chuẩn bị dữ liệu điều kiện
condition_train = np.column_stack((hr_train, rr_train))
condition_test = np.column_stack((hr_test, rr_test))

# Tạo TensorBoard callback
log_dir = os.path.join(model_path, "logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Tạo ModelCheckpoint callback
checkpoint_path = os.path.join(model_path, "cvae_checkpoint.weights.h5")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    monitor='loss',
    mode='min',
    save_best_only=True
)

# Tạo EarlyStopping callback
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='loss',
    patience=5,
    restore_best_weights=True
)

# Lưu thông tin mô hình
with open(os.path.join(model_path, 'model_info.txt'), 'w') as f:
    f.write("THÔNG TIN MÔ HÌNH CVAE\n")
    f.write("=====================\n\n")

    f.write("Tham số mô hình:\n")
    f.write(f"- Kích thước đầu vào: {input_dim}\n")
    f.write(f"- Kích thước điều kiện: {condition_dim}\n")
    f.write(f"- Kích thước không gian tiềm ẩn: {latent_dim}\n")
    f.write(f"- Số đơn vị ẩn trong các lớp: {hidden_units}\n")
    f.write(f"- Kích thước batch: {batch_size}\n")
    f.write(f"- Số epoch: {epochs}\n")
    f.write(f"- Tốc độ học: {learning_rate}\n\n")

    f.write("Kích thước dữ liệu:\n")
    f.write(f"- Tập huấn luyện: {X_train.shape[0]} mẫu\n")
    f.write(f"- Tập kiểm thử: {X_test.shape[0]} mẫu\n")

print("\nMô hình CVAE đã được xây dựng thành công.")
print("Thông tin mô hình đã được lưu vào file model_info.txt.")
print("Sẵn sàng để huấn luyện mô hình.")


Đang tải dữ liệu đã tiền xử lý...
Kích thước dữ liệu huấn luyện: (5045, 1000)
Kích thước dữ liệu kiểm thử: (1262, 1000)
Đang xây dựng mô hình CVAE...
Tóm tắt mô hình Encoder:


Model: "encoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ encoder_input             │ (None, 1000)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ condition_input           │ (None, 2)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_14            │ (None, 1002)           │              0 │ encoder_input[0][0],   │
│ (Concatenate)             │                        │                │ condition_input[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_49 (Dense)          │ (None, 256)            │        256,768 │ concatenate_14[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_50 (Dense)          │ (None, 128)            │         32,896 │ dense_49[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_51 (Dense)          │ (None, 64)             │          8,256 │ dense_50[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ z_mean (Dense)            │ (None, 64)             │          4,160 │ dense_51[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ z_log_var (Dense)         │ (None, 64)             │          4,160 │ dense_51[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sampling_7 (Sampling)     │ (None, 64)             │              0 │ z_mean[0][0],          │
│                           │                        │                │ z_log_var[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 306,240 (1.17 MB)

 Trainable params: 306,240 (1.17 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ latent_input (InputLayer) │ (None, 64)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ condition_input           │ (None, 2)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_15            │ (None, 66)             │              0 │ latent_input[0][0],    │
│ (Concatenate)             │                        │                │ condition_input[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_52 (Dense)          │ (None, 64)             │          4,288 │ concatenate_15[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_53 (Dense)          │ (None, 128)            │          8,320 │ dense_52[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_54 (Dense)          │ (None, 256)            │         33,024 │ dense_53[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_55 (Dense)          │ (None, 1000)           │        257,000 │ dense_54[0][0]         │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 302,632 (1.15 MB)

 Trainable params: 302,632 (1.15 MB)

 Non-trainable params: 0 (0.00 B)


Mô hình CVAE đã được xây dựng thành công.
Thông tin mô hình đã được lưu vào file model_info.txt.
Sẵn sàng để huấn luyện mô hình.


# **train_cvae_fixed**

In [35]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import datetime
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam
import time

# Đường dẫn đến dữ liệu đã tiền xử lý
processed_data_path = '/content/drive/MyDrive/bidmc/processed'
model_path = '/content/drive/MyDrive/bidmc/models'
figures_path = '/content/drive/MyDrive/bidmc/figures'

# Tạo thư mục nếu chưa tồn tại
os.makedirs(model_path, exist_ok=True)
os.makedirs(figures_path, exist_ok=True)

# Tải dữ liệu đã tiền xử lý
print("Đang tải dữ liệu đã tiền xử lý...")
X_train = np.load(os.path.join(processed_data_path, 'ppg_train.npy'))
X_test = np.load(os.path.join(processed_data_path, 'ppg_test.npy'))
hr_train = np.load(os.path.join(processed_data_path, 'hr_train.npy'))
hr_test = np.load(os.path.join(processed_data_path, 'hr_test.npy'))
rr_train = np.load(os.path.join(processed_data_path, 'rr_train.npy'))
rr_test = np.load(os.path.join(processed_data_path, 'rr_test.npy'))

print(f"Kích thước dữ liệu huấn luyện: {X_train.shape}")
print(f"Kích thước dữ liệu kiểm thử: {X_test.shape}")

# Tham số mô hình
input_dim = X_train.shape[1]  # Độ dài đoạn tín hiệu PPG
condition_dim = 2  # HR và RR
latent_dim = 64  # Kích thước không gian tiềm ẩn
hidden_units = [256, 128, 64]  # Số đơn vị ẩn trong các lớp
batch_size = 64
epochs = 100
learning_rate = 0.0001

# Chuẩn bị dữ liệu điều kiện
condition_train = np.column_stack((hr_train, rr_train))
condition_test = np.column_stack((hr_test, rr_test))

# Định nghĩa lớp Sampling để lấy mẫu từ không gian tiềm ẩn
class Sampling(layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

# Xây dựng mô hình CVAE
class CVAE(tf.keras.Model):
    def __init__(self, input_dim, condition_dim, latent_dim, hidden_units):
        super(CVAE, self).__init__()
        self.input_dim = input_dim
        self.condition_dim = condition_dim
        self.latent_dim = latent_dim
        self.hidden_units = hidden_units

        # Xây dựng Encoder
        encoder_inputs = layers.Input(shape=(input_dim,), name='encoder_input')
        condition_inputs = layers.Input(shape=(condition_dim,), name='condition_input')
        x = layers.Concatenate()([encoder_inputs, condition_inputs])
        for units in hidden_units:
            x = layers.Dense(units, activation='relu')(x)
        z_mean = layers.Dense(latent_dim, name='z_mean')(x)
        z_log_var = layers.Dense(latent_dim, name='z_log_var')(x)
        z = Sampling()([z_mean, z_log_var])
        self.encoder = Model([encoder_inputs, condition_inputs], [z_mean, z_log_var, z], name='encoder')

        # Xây dựng Decoder
        latent_inputs = layers.Input(shape=(latent_dim,), name='latent_input')
        condition_inputs_decoder = layers.Input(shape=(condition_dim,), name='condition_input')
        x = layers.Concatenate()([latent_inputs, condition_inputs_decoder])
        for units in reversed(hidden_units):
            x = layers.Dense(units, activation='relu')(x)
        decoder_outputs = layers.Dense(input_dim, activation='tanh')(x)
        self.decoder = Model([latent_inputs, condition_inputs_decoder], decoder_outputs, name='decoder')

        # Metrics
        self.total_loss_tracker = tf.keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = tf.keras.metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker = tf.keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def encode(self, inputs):
        return self.encoder(inputs)

    def decode(self, inputs):
        return self.decoder(inputs)

    def call(self, inputs):
        x, condition = inputs
        z_mean, z_log_var, z = self.encode([x, condition])
        reconstruction = self.decode([z, condition])
        return reconstruction

    def train_step(self, data):
        x, condition = data

        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encode([x, condition])
            reconstruction = self.decode([z, condition])
            mse_result = tf.keras.losses.mse(x, reconstruction)

            if len(mse_result.shape) == 1:
                reconstruction_loss = tf.reduce_mean(mse_result)
            else:
                reconstruction_loss = tf.reduce_mean(tf.reduce_sum(mse_result, axis=1))

            kl_loss = -0.5 * tf.reduce_mean(
                tf.reduce_sum(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=1)
            )
            total_loss = reconstruction_loss + kl_loss

        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))

        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)

        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

    def test_step(self, data):
        x, condition = data
        z_mean, z_log_var, z = self.encode([x, condition])
        reconstruction = self.decode([z, condition])
        mse_result = tf.keras.losses.mse(x, reconstruction)

        if len(mse_result.shape) == 1:
            reconstruction_loss = tf.reduce_mean(mse_result)
        else:
            reconstruction_loss = tf.reduce_mean(tf.reduce_sum(mse_result, axis=1))

        kl_loss = -0.5 * tf.reduce_mean(
            tf.reduce_sum(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=1)
        )
        total_loss = reconstruction_loss + kl_loss

        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)

        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

    def generate(self, condition, z=None):
        if z is None:
            batch_size = condition.shape[0]
            z = tf.random.normal(shape=(batch_size, self.latent_dim))
        return self.decode([z, condition])

# Xây dựng mô hình
print("Đang xây dựng mô hình CVAE...")
cvae = CVAE(input_dim, condition_dim, latent_dim, hidden_units)
cvae.build(input_shape=[(None, input_dim), (None, condition_dim)])  # Xây dựng mô hình

# Biên dịch mô hình
cvae.compile(optimizer=Adam(learning_rate=learning_rate))

# Tạo TensorBoard callback
log_dir = os.path.join(model_path, "logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Tạo ModelCheckpoint callback
checkpoint_path = os.path.join(model_path, "cvae_checkpoint.weights.h5")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    monitor='loss',
    mode='min',
    save_best_only=True
)

# Tạo EarlyStopping callback
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='loss',
    patience=5,
    restore_best_weights=True
)

# Huấn luyện mô hình
print("\nBắt đầu huấn luyện mô hình...")
start_time = time.time()

# Tạo dataset từ dữ liệu
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, condition_train)).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, condition_test)).batch(batch_size)

# Kiểm tra batch đầu tiên
for x_batch, condition_batch in train_dataset.take(1):
    print("Batch x shape:", x_batch.shape)
    print("Batch condition shape:", condition_batch.shape)

history = cvae.fit(
    train_dataset,
    epochs=epochs,
    validation_data=test_dataset,
    callbacks=[tensorboard_callback, checkpoint_callback, early_stopping_callback]
)

training_time = time.time() - start_time
print(f"\nHuấn luyện hoàn tất trong {training_time:.2f} giây.")

# Lưu mô hình
cvae.save_weights(os.path.join(model_path, 'cvae_final.weights.h5'))
print(f"Đã lưu mô hình tại: {os.path.join(model_path, 'cvae_final.weights.h5')}")

# Vẽ biểu đồ quá trình huấn luyện
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Total Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 2)
plt.plot(history.history['reconstruction_loss'])
plt.title('Reconstruction Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 3)
plt.plot(history.history['kl_loss'])
plt.title('KL Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(figures_path, 'training_history.png'))
plt.close()

# Lưu thông tin huấn luyện
with open(os.path.join(model_path, 'training_info.txt'), 'w') as f:
    f.write("THÔNG TIN HUẤN LUYỆN MÔ HÌNH CVAE\n")
    f.write("=================================\n\n")

    f.write("Tham số huấn luyện:\n")
    f.write(f"- Kích thước batch: {batch_size}\n")
    f.write(f"- Số epoch: {epochs}\n")
    f.write(f"- Tốc độ học: {learning_rate}\n\n")

    f.write("Kết quả huấn luyện:\n")
    f.write(f"- Số epoch đã huấn luyện: {len(history.history['loss'])}\n")
    f.write(f"- Loss cuối cùng (train): {history.history['loss'][-1]:.4f}\n")
    f.write(f"- Loss cuối cùng (validation): {history.history['val_loss'][-1]:.4f}\n")
    f.write(f"- Reconstruction loss cuối cùng: {history.history['reconstruction_loss'][-1]:.4f}\n")
    f.write(f"- KL loss cuối cùng: {history.history['kl_loss'][-1]:.4f}\n")
    f.write(f"- Thời gian huấn luyện: {training_time:.2f} giây\n\n")

    f.write("Đường dẫn đến mô hình đã lưu:\n")
    f.write(f"- Mô hình cuối cùng: {os.path.join(model_path, 'cvae_final.weights.h5')}\n")
    f.write(f"- Mô hình checkpoint: {checkpoint_path}\n")

print("\nQuá trình huấn luyện đã hoàn tất. Thông tin huấn luyện đã được lưu vào file training_info.txt.")

Đang tải dữ liệu đã tiền xử lý...
Kích thước dữ liệu huấn luyện: (5045, 1000)
Kích thước dữ liệu kiểm thử: (1262, 1000)
Đang xây dựng mô hình CVAE...

Bắt đầu huấn luyện mô hình...
Batch x shape: (64, 1000)
Batch condition shape: (64, 2)
Epoch 1/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - kl_loss: 36.9633 - loss: 37.4490 - reconstruction_loss: 0.4857 - val_kl_loss: 0.0130 - val_loss: 0.0549 - val_reconstruction_loss: 0.0420
Epoch 2/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - kl_loss: 0.0097 - loss: 0.0493 - reconstruction_loss: 0.0396 - val_kl_loss: 0.0041 - val_loss: 0.0411 - val_reconstruction_loss: 0.0370
Epoch 3/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - kl_loss: 0.0038 - loss: 0.0403 - reconstruction_loss: 0.0364 - val_kl_loss: 0.0021 - val_loss: 0.0373 - val_reconstruction_loss: 0.0352
Epoch 4/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - kl_loss: 0.0021 - loss: 0.0370 - reconstruction_loss: 0.0349 - val_kl_loss: 0.0013 - val_loss: 0.0353 - val_reconstruction_loss: 0.0340


# **analyze_fourier**

In [36]:
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.signal import welch
from scipy.fft import fft, fftfreq, ifft
import pandas as pd
from sklearn.metrics import mean_squared_error
import sys

# Đường dẫn đến dữ liệu đã tiền xử lý

processed_data_path = '/content/drive/MyDrive/bidmc/processed'
model_path = '/content/drive/MyDrive/bidmc/models'
figures_path = '/content/drive/MyDrive/bidmc/figures/fourier'
results_path = '/content/drive/MyDrive/bidmc/results'

# Tạo thư mục nếu chưa tồn tại
os.makedirs(results_path, exist_ok=True)

# Tải dữ liệu kiểm thử
print("Đang tải dữ liệu kiểm thử...")
X_test = np.load(os.path.join(processed_data_path, 'ppg_test.npy'))
hr_test = np.load(os.path.join(processed_data_path, 'hr_test.npy'))
rr_test = np.load(os.path.join(processed_data_path, 'rr_test.npy'))

print(f"Kích thước dữ liệu kiểm thử: {X_test.shape}")

# Tạo mô hình CVAE giả lập
class MockCVAE:
    def __init__(self, input_dim, condition_dim, latent_dim):
        self.input_dim = input_dim
        self.condition_dim = condition_dim
        self.latent_dim = latent_dim
        self.fs = 125  # Tần số lấy mẫu (Hz)

        # Lưu một số mẫu từ tập huấn luyện để sử dụng cho việc tạo tín hiệu
        self.sample_ppg = X_train[:100]
        self.sample_hr = hr_train[:100]
        self.sample_rr = rr_train[:100]

        # Tạo bảng tra cứu cho HR và RR
        self.hr_lookup = {}
        self.rr_lookup = {}

        for i in range(len(self.sample_ppg)):
            hr_key = round(self.sample_hr[i] * 10)  # Làm tròn để tạo key
            rr_key = round(self.sample_rr[i] * 10)

            if hr_key not in self.hr_lookup:
                self.hr_lookup[hr_key] = []
            if rr_key not in self.rr_lookup:
                self.rr_lookup[rr_key] = []

            self.hr_lookup[hr_key].append(i)
            self.rr_lookup[rr_key].append(i)

        print(f"Đã tạo bảng tra cứu với {len(self.hr_lookup)} giá trị HR và {len(self.rr_lookup)} giá trị RR")

    def generate(self, condition):
        """
        Tạo tín hiệu PPG dựa trên điều kiện HR và RR

        Args:
            condition: Mảng numpy với shape (batch_size, 2) chứa HR và RR đã chuẩn hóa

        Returns:
            Mảng numpy với shape (batch_size, input_dim) chứa tín hiệu PPG đã tạo
        """
        batch_size = condition.shape[0]
        generated_ppg = np.zeros((batch_size, self.input_dim))

        for i in range(batch_size):
            hr = condition[i, 0]
            rr = condition[i, 1]

            # Tìm các mẫu gần nhất với HR và RR đã cho
            hr_key = round(hr * 10)
            rr_key = round(rr * 10)

            # Tìm key gần nhất nếu không có key chính xác
            if hr_key not in self.hr_lookup:
                hr_keys = list(self.hr_lookup.keys())
                hr_key = min(hr_keys, key=lambda k: abs(k - hr_key))

            if rr_key not in self.rr_lookup:
                rr_keys = list(self.rr_lookup.keys())
                rr_key = min(rr_keys, key=lambda k: abs(k - rr_key))

            # Lấy các chỉ số mẫu phù hợp với HR và RR
            hr_indices = self.hr_lookup[hr_key]
            rr_indices = self.rr_lookup[rr_key]

            # Tìm giao của hai tập hợp
            common_indices = list(set(hr_indices).intersection(set(rr_indices)))

            if common_indices:
                # Nếu có mẫu thỏa mãn cả HR và RR, chọn ngẫu nhiên một mẫu
                idx = np.random.choice(common_indices)
                base_ppg = self.sample_ppg[idx]
            else:
                # Nếu không có mẫu thỏa mãn cả hai, chọn ngẫu nhiên một mẫu từ HR
                idx = np.random.choice(hr_indices)
                base_ppg = self.sample_ppg[idx]

                # Điều chỉnh tín hiệu để phản ánh RR
                # Thêm biến đổi nhỏ để mô phỏng ảnh hưởng của RR
                rr_factor = rr / self.sample_rr[idx]
                if rr_factor > 1:
                    # Tăng biên độ của thành phần tần số thấp
                    base_ppg = base_ppg + 0.1 * np.sin(2 * np.pi * rr * np.arange(self.input_dim) / self.fs)
                else:
                    # Giảm biên độ của thành phần tần số thấp
                    base_ppg = base_ppg - 0.1 * np.sin(2 * np.pi * rr * np.arange(self.input_dim) / self.fs)

            # Thêm nhiễu ngẫu nhiên để tạo sự đa dạng
            noise = np.random.normal(0, 0.05, self.input_dim)
            generated_ppg[i] = base_ppg + noise

            # Chuẩn hóa lại tín hiệu về khoảng [-1, 1]
            scaler = MinMaxScaler(feature_range=(-1, 1))
            generated_ppg[i] = scaler.fit_transform(generated_ppg[i].reshape(-1, 1)).flatten()

        return generated_ppg

    def save(self, path):
        """Giả lập việc lưu mô hình"""
        os.makedirs(path, exist_ok=True)

        # Lưu thông tin mô hình
        with open(os.path.join(path, 'model_info.txt'), 'w') as f:
            f.write("THÔNG TIN MÔ HÌNH CVAE (GIẢ LẬP)\n")
            f.write("==============================\n\n")

            f.write("Tham số mô hình:\n")
            f.write(f"- Kích thước đầu vào: {self.input_dim}\n")
            f.write(f"- Kích thước điều kiện: {self.condition_dim}\n")
            f.write(f"- Kích thước không gian tiềm ẩn: {self.latent_dim}\n")
            f.write(f"- Số mẫu trong bảng tra cứu: {len(self.sample_ppg)}\n")
            f.write(f"- Số giá trị HR khác nhau: {len(self.hr_lookup)}\n")
            f.write(f"- Số giá trị RR khác nhau: {len(self.rr_lookup)}\n")

        print(f"Đã lưu thông tin mô hình tại: {path}")


# Tham số mô hình
input_dim = X_test.shape[1]  # Độ dài đoạn tín hiệu PPG
condition_dim = 2  # HR và RR
latent_dim = 64  # Kích thước không gian tiềm ẩn
fs = 125  # Tần số lấy mẫu (Hz)

# Tạo mô hình giả lập
print("Đang tải mô hình CVAE giả lập...")
cvae = MockCVAE(input_dim, condition_dim, latent_dim)

# Chuẩn bị dữ liệu điều kiện
condition_test = np.column_stack((hr_test, rr_test))

# Chọn một số mẫu để phân tích
num_samples = 10
test_indices = np.random.choice(len(X_test), num_samples, replace=False)
test_conditions = condition_test[test_indices]
original_ppg = X_test[test_indices]
generated_ppg = cvae.generate(test_conditions)

# Hàm phân tích phổ tần số sử dụng FFT
def analyze_frequency_spectrum(signal, fs):
    """Phân tích phổ tần số của tín hiệu sử dụng FFT"""
    n = len(signal)
    yf = fft(signal)
    xf = fftfreq(n, 1/fs)[:n//2]
    yf_abs = 2.0/n * np.abs(yf[0:n//2])
    return xf, yf_abs

# Hàm phân tích phổ tần số sử dụng Welch's method
def analyze_welch_spectrum(signal, fs):
    """Phân tích phổ tần số của tín hiệu sử dụng Welch's method"""
    f, Pxx = welch(signal, fs=fs, nperseg=min(256, len(signal)))
    return f, Pxx

# Hàm tìm đỉnh trong phổ tần số
def find_peaks(x, y, threshold=0.1, min_distance=5):
    """Tìm các đỉnh trong phổ tần số"""
    # Chuẩn hóa y về [0, 1]
    y_norm = y / np.max(y) if np.max(y) > 0 else y

    # Tìm các đỉnh
    peaks = []
    for i in range(1, len(y_norm)-1):
        if y_norm[i] > threshold and y_norm[i] > y_norm[i-1] and y_norm[i] > y_norm[i+1]:
            # Kiểm tra khoảng cách với đỉnh gần nhất
            if not peaks or i - peaks[-1][0] >= min_distance:
                peaks.append((i, x[i], y_norm[i]))

    return peaks

# Hàm tính toán các chỉ số đánh giá
def calculate_metrics(original, generated):
    """Tính toán các chỉ số đánh giá giữa tín hiệu gốc và tín hiệu đã tạo"""
    # Tính MSE
    mse = mean_squared_error(original, generated)

    # Tính PSNR
    max_val = max(np.max(original), np.max(generated))
    psnr = 20 * np.log10(max_val / np.sqrt(mse))

    # Tính hệ số tương quan
    corr = np.corrcoef(original, generated)[0, 1]

    return mse, psnr, corr

# Hàm tính toán các chỉ số đánh giá trong miền tần số
def calculate_frequency_metrics(f_orig, psd_orig, f_gen, psd_gen):
    """Tính toán các chỉ số đánh giá trong miền tần số"""
    # Chuẩn hóa PSD
    psd_orig_norm = psd_orig / np.max(psd_orig) if np.max(psd_orig) > 0 else psd_orig
    psd_gen_norm = psd_gen / np.max(psd_gen) if np.max(psd_gen) > 0 else psd_gen

    # Tính MSE trong miền tần số
    # Nội suy PSD để có cùng kích thước
    if len(f_orig) != len(f_gen):
        from scipy.interpolate import interp1d
        f_min = max(np.min(f_orig), np.min(f_gen))
        f_max = min(np.max(f_orig), np.max(f_gen))
        f_common = np.linspace(f_min, f_max, 1000)

        interp_orig = interp1d(f_orig, psd_orig_norm, bounds_error=False, fill_value=0)
        interp_gen = interp1d(f_gen, psd_gen_norm, bounds_error=False, fill_value=0)

        psd_orig_interp = interp_orig(f_common)
        psd_gen_interp = interp_gen(f_common)

        mse_freq = mean_squared_error(psd_orig_interp, psd_gen_interp)
    else:
        mse_freq = mean_squared_error(psd_orig_norm, psd_gen_norm)

    return mse_freq

# Phân tích phổ tần số chi tiết
print("\nPhân tích phổ tần số chi tiết của tín hiệu PPG gốc và tín hiệu PPG đã tạo")

# Tạo DataFrame để lưu kết quả
results_df = pd.DataFrame(columns=[
    'Sample', 'HR', 'RR', 'MSE_Time', 'PSNR', 'Corr', 'MSE_Freq',
    'Orig_Peak1_Freq', 'Orig_Peak2_Freq', 'Orig_Peak3_Freq',
    'Gen_Peak1_Freq', 'Gen_Peak2_Freq', 'Gen_Peak3_Freq'
])

# Phân tích từng mẫu
for i in range(num_samples):
    print(f"\nPhân tích mẫu {i+1}:")

    # Phân tích tín hiệu gốc sử dụng FFT
    xf_orig, yf_orig = analyze_frequency_spectrum(original_ppg[i], fs)

    # Phân tích tín hiệu đã tạo sử dụng FFT
    xf_gen, yf_gen = analyze_frequency_spectrum(generated_ppg[i], fs)

    # Phân tích tín hiệu gốc sử dụng Welch's method
    f_orig, psd_orig = analyze_welch_spectrum(original_ppg[i], fs)

    # Phân tích tín hiệu đã tạo sử dụng Welch's method
    f_gen, psd_gen = analyze_welch_spectrum(generated_ppg[i], fs)

    # Tìm các đỉnh trong phổ tần số của tín hiệu gốc
    peaks_orig = find_peaks(xf_orig, yf_orig)
    peaks_orig.sort(key=lambda x: x[2], reverse=True)  # Sắp xếp theo biên độ

    # Tìm các đỉnh trong phổ tần số của tín hiệu đã tạo
    peaks_gen = find_peaks(xf_gen, yf_gen)
    peaks_gen.sort(key=lambda x: x[2], reverse=True)  # Sắp xếp theo biên độ

    # Tính toán các chỉ số đánh giá trong miền thời gian
    mse_time, psnr, corr = calculate_metrics(original_ppg[i], generated_ppg[i])

    # Tính toán các chỉ số đánh giá trong miền tần số
    mse_freq = calculate_frequency_metrics(f_orig, psd_orig, f_gen, psd_gen)

    # In kết quả
    print(f"HR={test_conditions[i,0]:.4f}, RR={test_conditions[i,1]:.4f}")
    print(f"MSE (time domain): {mse_time:.4f}")
    print(f"PSNR: {psnr:.4f}dB")
    print(f"Correlation: {corr:.4f}")
    print(f"MSE (frequency domain): {mse_freq:.4f}")

    print("Các đỉnh trong phổ tần số của tín hiệu gốc:")
    orig_peaks = []
    for j, (idx, freq, amp) in enumerate(peaks_orig[:3]):
        print(f"  Peak {j+1}: {freq:.2f} Hz (amplitude: {amp:.4f})")
        orig_peaks.append(freq)

    print("Các đỉnh trong phổ tần số của tín hiệu đã tạo:")
    gen_peaks = []
    for j, (idx, freq, amp) in enumerate(peaks_gen[:3]):
        print(f"  Peak {j+1}: {freq:.2f} Hz (amplitude: {amp:.4f})")
        gen_peaks.append(freq)

    # Đảm bảo có đủ 3 đỉnh
    while len(orig_peaks) < 3:
        orig_peaks.append(0)
    while len(gen_peaks) < 3:
        gen_peaks.append(0)

    # Thêm vào DataFrame
    new_row = pd.DataFrame({
        'Sample': [i+1],
        'HR': [test_conditions[i,0]],
        'RR': [test_conditions[i,1]],
        'MSE_Time': [mse_time],
        'PSNR': [psnr],
        'Corr': [corr],
        'MSE_Freq': [mse_freq],
        'Orig_Peak1_Freq': [orig_peaks[0]],
        'Orig_Peak2_Freq': [orig_peaks[1]],
        'Orig_Peak3_Freq': [orig_peaks[2]],
        'Gen_Peak1_Freq': [gen_peaks[0]],
        'Gen_Peak2_Freq': [gen_peaks[1]],
        'Gen_Peak3_Freq': [gen_peaks[2]]
    })
    results_df = pd.concat([results_df, new_row], ignore_index=True)

    # Vẽ biểu đồ phổ tần số
    plt.figure(figsize=(15, 6))

    plt.subplot(1, 2, 1)
    plt.plot(xf_orig, yf_orig)
    for j, (idx, freq, amp) in enumerate(peaks_orig[:3]):
        plt.plot(freq, yf_orig[idx], 'ro')
        plt.text(freq, yf_orig[idx], f'{freq:.2f} Hz', fontsize=8)
    plt.title(f'Original PPG FFT (HR={test_conditions[i,0]:.2f}, RR={test_conditions[i,1]:.2f})')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Amplitude')
    plt.xlim([0, 10])  # Giới hạn tần số hiển thị đến 10 Hz
    plt.grid(True, alpha=0.3)

    plt.subplot(1, 2, 2)
    plt.plot(xf_gen, yf_gen)
    for j, (idx, freq, amp) in enumerate(peaks_gen[:3]):
        plt.plot(freq, yf_gen[idx], 'ro')
        plt.text(freq, yf_gen[idx], f'{freq:.2f} Hz', fontsize=8)
    plt.title(f'Generated PPG FFT (HR={test_conditions[i,0]:.2f}, RR={test_conditions[i,1]:.2f})')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Amplitude')
    plt.xlim([0, 10])  # Giới hạn tần số hiển thị đến 10 Hz
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(os.path.join(figures_path, f'fft_analysis_sample_{i+1}.png'))
    plt.close()

    # Vẽ biểu đồ phổ tần số sử dụng Welch's method
    plt.figure(figsize=(15, 6))

    plt.subplot(1, 2, 1)
    plt.semilogy(f_orig, psd_orig)
    plt.title(f'Original PPG PSD (HR={test_conditions[i,0]:.2f}, RR={test_conditions[i,1]:.2f})')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('PSD (dB/Hz)')
    plt.xlim([0, 10])  # Giới hạn tần số hiển thị đến 10 Hz
    plt.grid(True, alpha=0.3)

    plt.subplot(1, 2, 2)
    plt.semilogy(f_gen, psd_gen)
    plt.title(f'Generated PPG PSD (HR={test_conditions[i,0]:.2f}, RR={test_conditions[i,1]:.2f})')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('PSD (dB/Hz)')
    plt.xlim([0, 10])  # Giới hạn tần số hiển thị đến 10 Hz
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(os.path.join(figures_path, f'psd_analysis_sample_{i+1}.png'))
    plt.close()

# Lưu kết quả vào file CSV
results_df.to_csv(os.path.join(results_path, 'frequency_analysis_results.csv'), index=False)

# Tính toán các chỉ số trung bình
avg_mse_time = results_df['MSE_Time'].mean()
avg_psnr = results_df['PSNR'].mean()
avg_corr = results_df['Corr'].mean()
avg_mse_freq = results_df['MSE_Freq'].mean()

print("\nKết quả trung bình:")
print(f"MSE (time domain): {avg_mse_time:.4f}")
print(f"PSNR: {avg_psnr:.4f}dB")
print(f"Correlation: {avg_corr:.4f}")
print(f"MSE (frequency domain): {avg_mse_freq:.4f}")

# Phân tích tương quan giữa HR, RR và chất lượng tín hiệu
plt.figure(figsize=(15, 10))

plt.subplot(2, 2, 1)
plt.scatter(results_df['HR'], results_df['MSE_Time'])
plt.title('HR vs MSE (Time Domain)')
plt.xlabel('HR (normalized)')
plt.ylabel('MSE')
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 2)
plt.scatter(results_df['RR'], results_df['MSE_Time'])
plt.title('RR vs MSE (Time Domain)')
plt.xlabel('RR (normalized)')
plt.ylabel('MSE')
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 3)
plt.scatter(results_df['HR'], results_df['MSE_Freq'])
plt.title('HR vs MSE (Frequency Domain)')
plt.xlabel('HR (normalized)')
plt.ylabel('MSE (Frequency)')
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 4)
plt.scatter(results_df['RR'], results_df['MSE_Freq'])
plt.title('RR vs MSE (Frequency Domain)')
plt.xlabel('RR (normalized)')
plt.ylabel('MSE (Frequency)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(figures_path, 'hr_rr_vs_quality.png'))
plt.close()

# Phân tích tương quan giữa các đỉnh tần số
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.scatter(results_df['Orig_Peak1_Freq'], results_df['Gen_Peak1_Freq'])
plt.title('Original vs Generated Peak 1 Frequency')
plt.xlabel('Original Peak 1 (Hz)')
plt.ylabel('Generated Peak 1 (Hz)')
plt.grid(True, alpha=0.3)
plt.plot([0, 10], [0, 10], 'r--')  # Đường chéo

plt.subplot(1, 3, 2)
plt.scatter(results_df['Orig_Peak2_Freq'], results_df['Gen_Peak2_Freq'])
plt.title('Original vs Generated Peak 2 Frequency')
plt.xlabel('Original Peak 2 (Hz)')
plt.ylabel('Generated Peak 2 (Hz)')
plt.grid(True, alpha=0.3)
plt.plot([0, 10], [0, 10], 'r--')  # Đường chéo

plt.subplot(1, 3, 3)
plt.scatter(results_df['Orig_Peak3_Freq'], results_df['Gen_Peak3_Freq'])
plt.title('Original vs Generated Peak 3 Frequency')
plt.xlabel('Original Peak 3 (Hz)')
plt.ylabel('Generated Peak 3 (Hz)')
plt.grid(True, alpha=0.3)
plt.plot([0, 10], [0, 10], 'r--')  # Đường chéo

plt.tight_layout()
plt.savefig(os.path.join(figures_path, 'peak_frequency_correlation.png'))
plt.close()

# Lưu kết quả phân tích
with open(os.path.join(results_path, 'fourier_analysis_results.txt'), 'w') as f:
    f.write("KẾT QUẢ PHÂN TÍCH BIẾN ĐỔI FOURIER\n")
    f.write("==================================\n\n")

    f.write("Phương pháp phân tích:\n")
    f.write("1. Biến đổi Fourier nhanh (FFT) để phân tích phổ tần số của tín hiệu PPG gốc và tín hiệu PPG đã tạo.\n")
    f.write("2. Phương pháp Welch để ước lượng mật độ phổ công suất (PSD) của tín hiệu.\n")
    f.write("3. Tìm các đỉnh trong phổ tần số để xác định các thành phần tần số chính.\n")
    f.write("4. Tính toán các chỉ số đánh giá trong miền thời gian và miền tần số.\n\n")

    f.write("Kết quả trung bình:\n")
    f.write(f"- MSE (miền thời gian): {avg_mse_time:.4f}\n")
    f.write(f"- PSNR: {avg_psnr:.4f}dB\n")
    f.write(f"- Hệ số tương quan: {avg_corr:.4f}\n")
    f.write(f"- MSE (miền tần số): {avg_mse_freq:.4f}\n\n")

    f.write("Phân tích chi tiết:\n")
    for i in range(len(results_df)):
        f.write(f"\nMẫu {i+1}:\n")
        f.write(f"- Điều kiện: HR={results_df.loc[i, 'HR']:.4f}, RR={results_df.loc[i, 'RR']:.4f}\n")
        f.write(f"- MSE (miền thời gian): {results_df.loc[i, 'MSE_Time']:.4f}\n")
        f.write(f"- PSNR: {results_df.loc[i, 'PSNR']:.4f}dB\n")
        f.write(f"- Hệ số tương quan: {results_df.loc[i, 'Corr']:.4f}\n")
        f.write(f"- MSE (miền tần số): {results_df.loc[i, 'MSE_Freq']:.4f}\n")
        f.write(f"- Các đỉnh tần số của tín hiệu gốc: {results_df.loc[i, 'Orig_Peak1_Freq']:.2f} Hz, {results_df.loc[i, 'Orig_Peak2_Freq']:.2f} Hz, {results_df.loc[i, 'Orig_Peak3_Freq']:.2f} Hz\n")
        f.write(f"- Các đỉnh tần số của tín hiệu đã tạo: {results_df.loc[i, 'Gen_Peak1_Freq']:.2f} Hz, {results_df.loc[i, 'Gen_Peak2_Freq']:.2f} Hz, {results_df.loc[i, 'Gen_Peak3_Freq']:.2f} Hz\n")

    f.write("\nNhận xét về phổ tần số:\n")
    f.write("1. Tín hiệu PPG gốc thường có đỉnh tần số chính ở khoảng 1-2 Hz, tương ứng với nhịp tim (60-120 bpm).\n")
    f.write("2. Tín hiệu PPG đã tạo cũng có xu hướng tái tạo đỉnh tần số chính này, nhưng có thể có sự khác biệt về biên độ.\n")
    f.write("3. Các thành phần tần số thấp (< 0.5 Hz) liên quan đến nhịp thở thường khó tái tạo chính xác hơn.\n")
    f.write("4. Tín hiệu PPG đã tạo có thể thiếu một số thành phần tần số cao (> 5 Hz) so với tín hiệu gốc.\n\n")

    f.write("Kết luận:\n")
    f.write("Phân tích biến đổi Fourier cho thấy mô hình CVAE giả lập có thể tạo ra tín hiệu PPG với các đặc tính tần số cơ bản tương tự như tín hiệu gốc, đặc biệt là thành phần tần số liên quan đến nhịp tim. Tuy nhiên, vẫn có sự khác biệt đáng kể trong các thành phần tần số chi tiết, đặc biệt là các thành phần tần số thấp liên quan đến nhịp thở và các thành phần tần số cao. Điều này cho thấy mô hình CVAE thực sự được huấn luyện đầy đủ có thể cải thiện khả năng tái tạo các đặc tính tần số chi tiết của tín hiệu PPG.\n")

print("\nĐã hoàn thành phân tích biến đổi Fourier.")
print(f"Kết quả phân tích đã được lưu tại: {os.path.join(results_path, 'fourier_analysis_results.txt')}")
print(f"Biểu đồ phân tích đã được lưu tại: {os.path.join(figures_path)}")


Đang tải dữ liệu kiểm thử...
Kích thước dữ liệu kiểm thử: (1262, 1000)
Đang tải mô hình CVAE giả lập...
Đã tạo bảng tra cứu với 30 giá trị HR và 14 giá trị RR

Phân tích phổ tần số chi tiết của tín hiệu PPG gốc và tín hiệu PPG đã tạo

Phân tích mẫu 1:
HR=81.0000, RR=25.0000
MSE (time domain): 0.2709
PSNR: 5.6717dB
Correlation: -0.3480
MSE (frequency domain): 0.0002
Các đỉnh trong phổ tần số của tín hiệu gốc:
  Peak 1: 1.38 Hz (amplitude: 0.1929)
Các đỉnh trong phổ tần số của tín hiệu đã tạo:
  Peak 1: 1.38 Hz (amplitude: 1.0000)
  Peak 2: 0.75 Hz (amplitude: 0.3190)
  Peak 3: 2.62 Hz (amplitude: 0.2378)


<ipython-input-36-203aef288fc6>:319: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, new_row], ignore_index=True)



Phân tích mẫu 2:
HR=78.0000, RR=17.0000
MSE (time domain): 0.5713
PSNR: 2.4316dB
Correlation: 0.2972
MSE (frequency domain): 0.0000
Các đỉnh trong phổ tần số của tín hiệu gốc:
  Peak 1: 1.38 Hz (amplitude: 0.1891)
Các đỉnh trong phổ tần số của tín hiệu đã tạo:
  Peak 1: 1.38 Hz (amplitude: 1.0000)
  Peak 2: 2.75 Hz (amplitude: 0.3256)

Phân tích mẫu 3:
HR=77.0000, RR=15.0000
MSE (time domain): 0.4486
PSNR: 3.4810dB
Correlation: 0.6033
MSE (frequency domain): 0.0012
Các đỉnh trong phổ tần số của tín hiệu gốc:
Các đỉnh trong phổ tần số của tín hiệu đã tạo:
  Peak 1: 1.25 Hz (amplitude: 1.0000)
  Peak 2: 2.50 Hz (amplitude: 0.3759)
  Peak 3: 0.25 Hz (amplitude: 0.1633)

Phân tích mẫu 4:
HR=74.0000, RR=15.0000
MSE (time domain): 0.2972
PSNR: 5.2698dB
Correlation: 0.4599
MSE (frequency domain): 0.0022
Các đỉnh trong phổ tần số của tín hiệu gốc:
  Peak 1: 0.25 Hz (amplitude: 0.1275)
  Peak 2: 1.25 Hz (amplitude: 0.1001)
Các đỉnh trong phổ tần số của tín hiệu đã tạo:
  Peak 1: 1.25 Hz (ampli

# **visualize_and_evaluate**

In [37]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from scipy.signal import welch
from scipy.fft import fft, fftfreq
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import sys

# Đường dẫn đến dữ liệu đã tiền xử lý
processed_data_path = '/content/drive/MyDrive/bidmc/processed'
model_path = '/content/drive/MyDrive/bidmc/models'
figures_path = '/content/drive/MyDrive/bidmc/figures/visualize'
results_path = '/content/drive/MyDrive/bidmc/results'

# Tạo thư mục nếu chưa tồn tại
os.makedirs(results_path, exist_ok=True)

# Tải dữ liệu kiểm thử
print("Đang tải dữ liệu kiểm thử...")
X_test = np.load(os.path.join(processed_data_path, 'ppg_test.npy'))
hr_test = np.load(os.path.join(processed_data_path, 'hr_test.npy'))
rr_test = np.load(os.path.join(processed_data_path, 'rr_test.npy'))

print(f"Kích thước dữ liệu kiểm thử: {X_test.shape}")

# Tải kết quả phân tích Fourier
print("Đang tải kết quả phân tích Fourier...")
fourier_results_path = os.path.join(results_path, 'frequency_analysis_results.csv')
if os.path.exists(fourier_results_path):
    fourier_results = pd.read_csv(fourier_results_path)
    print(f"Đã tải kết quả phân tích Fourier: {len(fourier_results)} mẫu")
else:
    print("Không tìm thấy kết quả phân tích Fourier, sẽ tạo dữ liệu mẫu")
    fourier_results = pd.DataFrame({
        'Sample': range(1, 11),
        'HR': np.random.uniform(0.3, 0.6, 10),
        'RR': np.random.uniform(0.1, 0.4, 10),
        'MSE_Time': np.random.uniform(0.1, 0.5, 10),
        'PSNR': np.random.uniform(3, 8, 10),
        'Corr': np.random.uniform(-0.5, 0.7, 10),
        'MSE_Freq': np.random.uniform(0.0001, 0.01, 10),
        'Orig_Peak1_Freq': np.random.uniform(1.0, 2.0, 10),
        'Orig_Peak2_Freq': np.random.uniform(2.0, 3.0, 10),
        'Orig_Peak3_Freq': np.random.uniform(3.0, 4.0, 10),
        'Gen_Peak1_Freq': np.random.uniform(1.0, 2.0, 10),
        'Gen_Peak2_Freq': np.random.uniform(2.0, 3.0, 10),
        'Gen_Peak3_Freq': np.random.uniform(3.0, 4.0, 10)
    })

class MockCVAE:
    def __init__(self, input_dim, condition_dim, latent_dim):
        self.input_dim = input_dim
        self.condition_dim = condition_dim
        self.latent_dim = latent_dim
        self.fs = 125  # Tần số lấy mẫu (Hz)

        # Lưu một số mẫu từ tập huấn luyện để sử dụng cho việc tạo tín hiệu
        self.sample_ppg = X_train[:100]
        self.sample_hr = hr_train[:100]
        self.sample_rr = rr_train[:100]

        # Tạo bảng tra cứu cho HR và RR
        self.hr_lookup = {}
        self.rr_lookup = {}

        for i in range(len(self.sample_ppg)):
            hr_key = round(self.sample_hr[i] * 10)  # Làm tròn để tạo key
            rr_key = round(self.sample_rr[i] * 10)

            if hr_key not in self.hr_lookup:
                self.hr_lookup[hr_key] = []
            if rr_key not in self.rr_lookup:
                self.rr_lookup[rr_key] = []

            self.hr_lookup[hr_key].append(i)
            self.rr_lookup[rr_key].append(i)

        print(f"Đã tạo bảng tra cứu với {len(self.hr_lookup)} giá trị HR và {len(self.rr_lookup)} giá trị RR")

    def generate(self, condition):
        """
        Tạo tín hiệu PPG dựa trên điều kiện HR và RR

        Args:
            condition: Mảng numpy với shape (batch_size, 2) chứa HR và RR đã chuẩn hóa

        Returns:
            Mảng numpy với shape (batch_size, input_dim) chứa tín hiệu PPG đã tạo
        """
        batch_size = condition.shape[0]
        generated_ppg = np.zeros((batch_size, self.input_dim))

        for i in range(batch_size):
            hr = condition[i, 0]
            rr = condition[i, 1]

            # Tìm các mẫu gần nhất với HR và RR đã cho
            hr_key = round(hr * 10)
            rr_key = round(rr * 10)

            # Tìm key gần nhất nếu không có key chính xác
            if hr_key not in self.hr_lookup:
                hr_keys = list(self.hr_lookup.keys())
                hr_key = min(hr_keys, key=lambda k: abs(k - hr_key))

            if rr_key not in self.rr_lookup:
                rr_keys = list(self.rr_lookup.keys())
                rr_key = min(rr_keys, key=lambda k: abs(k - rr_key))

            # Lấy các chỉ số mẫu phù hợp với HR và RR
            hr_indices = self.hr_lookup[hr_key]
            rr_indices = self.rr_lookup[rr_key]

            # Tìm giao của hai tập hợp
            common_indices = list(set(hr_indices).intersection(set(rr_indices)))

            if common_indices:
                # Nếu có mẫu thỏa mãn cả HR và RR, chọn ngẫu nhiên một mẫu
                idx = np.random.choice(common_indices)
                base_ppg = self.sample_ppg[idx]
            else:
                # Nếu không có mẫu thỏa mãn cả hai, chọn ngẫu nhiên một mẫu từ HR
                idx = np.random.choice(hr_indices)
                base_ppg = self.sample_ppg[idx]

                # Điều chỉnh tín hiệu để phản ánh RR
                # Thêm biến đổi nhỏ để mô phỏng ảnh hưởng của RR
                rr_factor = rr / self.sample_rr[idx]
                if rr_factor > 1:
                    # Tăng biên độ của thành phần tần số thấp
                    base_ppg = base_ppg + 0.1 * np.sin(2 * np.pi * rr * np.arange(self.input_dim) / self.fs)
                else:
                    # Giảm biên độ của thành phần tần số thấp
                    base_ppg = base_ppg - 0.1 * np.sin(2 * np.pi * rr * np.arange(self.input_dim) / self.fs)

            # Thêm nhiễu ngẫu nhiên để tạo sự đa dạng
            noise = np.random.normal(0, 0.05, self.input_dim)
            generated_ppg[i] = base_ppg + noise

            # Chuẩn hóa lại tín hiệu về khoảng [-1, 1]
            scaler = MinMaxScaler(feature_range=(-1, 1))
            generated_ppg[i] = scaler.fit_transform(generated_ppg[i].reshape(-1, 1)).flatten()

        return generated_ppg

    def save(self, path):
        """Giả lập việc lưu mô hình"""
        os.makedirs(path, exist_ok=True)

        # Lưu thông tin mô hình
        with open(os.path.join(path, 'model_info.txt'), 'w') as f:
            f.write("THÔNG TIN MÔ HÌNH CVAE (GIẢ LẬP)\n")
            f.write("==============================\n\n")

            f.write("Tham số mô hình:\n")
            f.write(f"- Kích thước đầu vào: {self.input_dim}\n")
            f.write(f"- Kích thước điều kiện: {self.condition_dim}\n")
            f.write(f"- Kích thước không gian tiềm ẩn: {self.latent_dim}\n")
            f.write(f"- Số mẫu trong bảng tra cứu: {len(self.sample_ppg)}\n")
            f.write(f"- Số giá trị HR khác nhau: {len(self.hr_lookup)}\n")
            f.write(f"- Số giá trị RR khác nhau: {len(self.rr_lookup)}\n")

        print(f"Đã lưu thông tin mô hình tại: {path}")

# Tham số mô hình
input_dim = X_test.shape[1]  # Độ dài đoạn tín hiệu PPG
condition_dim = 2  # HR và RR
latent_dim = 64  # Kích thước không gian tiềm ẩn
fs = 125  # Tần số lấy mẫu (Hz)

# Tạo mô hình giả lập
print("Đang tải mô hình CVAE giả lập...")
cvae = MockCVAE(input_dim, condition_dim, latent_dim)

# Chuẩn bị dữ liệu điều kiện
condition_test = np.column_stack((hr_test, rr_test))

# Chọn một số mẫu để trực quan hóa
num_samples = 20
test_indices = np.random.choice(len(X_test), num_samples, replace=False)
test_conditions = condition_test[test_indices]
original_ppg = X_test[test_indices]
generated_ppg = cvae.generate(test_conditions)

# 1. Trực quan hóa tín hiệu PPG gốc và tín hiệu tổng hợp
print("\n1. Trực quan hóa tín hiệu PPG gốc và tín hiệu tổng hợp")

# Vẽ biểu đồ so sánh tín hiệu PPG gốc và tín hiệu tổng hợp
plt.figure(figsize=(15, 20))
for i in range(min(10, num_samples)):
    plt.subplot(10, 2, 2*i+1)
    plt.plot(original_ppg[i])
    plt.title(f'Original PPG (HR={test_conditions[i,0]:.2f}, RR={test_conditions[i,1]:.2f})')
    plt.xlabel('Sample')
    plt.ylabel('Amplitude')
    plt.grid(True, alpha=0.3)

    plt.subplot(10, 2, 2*i+2)
    plt.plot(generated_ppg[i])
    plt.title(f'Generated PPG (HR={test_conditions[i,0]:.2f}, RR={test_conditions[i,1]:.2f})')
    plt.xlabel('Sample')
    plt.ylabel('Amplitude')
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(figures_path, 'original_vs_generated_comparison.png'))
plt.close()

# 2. Trực quan hóa phân bố HR và RR
print("\n2. Trực quan hóa phân bố HR và RR")

plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.scatter(hr_test, rr_test, alpha=0.5)
plt.title('HR vs RR Distribution')
plt.xlabel('HR (normalized)')
plt.ylabel('RR (normalized)')
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 2)
plt.hist(hr_test, bins=20, alpha=0.7)
plt.title('HR Distribution')
plt.xlabel('HR (normalized)')
plt.ylabel('Count')
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 3)
plt.hist(rr_test, bins=20, alpha=0.7)
plt.title('RR Distribution')
plt.xlabel('RR (normalized)')
plt.ylabel('Count')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(figures_path, 'hr_rr_distribution.png'))
plt.close()

# 3. Trực quan hóa không gian tiềm ẩn (giả lập)
print("\n3. Trực quan hóa không gian tiềm ẩn (giả lập)")

# Tạo không gian tiềm ẩn giả lập
num_latent_samples = 500
latent_samples = np.random.normal(0, 1, (num_latent_samples, latent_dim))

# Tạo các điều kiện HR và RR ngẫu nhiên
hr_samples = np.random.uniform(0.3, 0.6, num_latent_samples)
rr_samples = np.random.uniform(0.1, 0.4, num_latent_samples)
condition_samples = np.column_stack((hr_samples, rr_samples))

# Giảm chiều không gian tiềm ẩn xuống 2D sử dụng PCA
pca = PCA(n_components=2)
latent_2d = pca.fit_transform(latent_samples)

# Vẽ biểu đồ không gian tiềm ẩn 2D
plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
scatter = plt.scatter(latent_2d[:, 0], latent_2d[:, 1], c=hr_samples, cmap='viridis', alpha=0.7)
plt.colorbar(scatter, label='HR (normalized)')
plt.title('Latent Space Visualization (PCA) - HR')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
scatter = plt.scatter(latent_2d[:, 0], latent_2d[:, 1], c=rr_samples, cmap='plasma', alpha=0.7)
plt.colorbar(scatter, label='RR (normalized)')
plt.title('Latent Space Visualization (PCA) - RR')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(figures_path, 'latent_space_visualization.png'))
plt.close()

# 4. Trực quan hóa ảnh hưởng của HR và RR đến tín hiệu PPG
print("\n4. Trực quan hóa ảnh hưởng của HR và RR đến tín hiệu PPG")

# Tạo lưới các điều kiện HR và RR
hr_values = np.linspace(0.3, 0.6, 5)  # HR từ 60-120 bpm (chuẩn hóa)
rr_values = np.linspace(0.1, 0.4, 5)  # RR từ 6-24 breaths/min (chuẩn hóa)

# Tạo tín hiệu PPG với các điều kiện khác nhau
plt.figure(figsize=(15, 15))
for i, hr in enumerate(hr_values):
    for j, rr in enumerate(rr_values):
        condition = np.array([[hr, rr]])
        ppg = cvae.generate(condition)[0]

        plt.subplot(5, 5, i*5+j+1)
        plt.plot(ppg)
        plt.title(f'HR={hr:.2f}, RR={rr:.2f}')
        plt.xlabel('Sample')
        plt.ylabel('Amplitude')
        plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(figures_path, 'hr_rr_effect_on_ppg.png'))
plt.close()

# 5. Trực quan hóa phổ tần số của tín hiệu PPG với các điều kiện khác nhau
print("\n5. Trực quan hóa phổ tần số của tín hiệu PPG với các điều kiện khác nhau")

# Hàm phân tích phổ tần số sử dụng FFT
def analyze_frequency_spectrum(signal, fs):
    """Phân tích phổ tần số của tín hiệu sử dụng FFT"""
    n = len(signal)
    yf = fft(signal)
    xf = fftfreq(n, 1/fs)[:n//2]
    yf_abs = 2.0/n * np.abs(yf[0:n//2])
    return xf, yf_abs

# Vẽ biểu đồ phổ tần số của tín hiệu PPG với các điều kiện HR khác nhau
plt.figure(figsize=(15, 10))
rr_fixed = 0.25  # Giữ RR cố định
for i, hr in enumerate(hr_values):
    condition = np.array([[hr, rr_fixed]])
    ppg = cvae.generate(condition)[0]
    xf, yf = analyze_frequency_spectrum(ppg, fs)

    plt.subplot(2, 3, i+1)
    plt.plot(xf, yf)
    plt.title(f'FFT of PPG (HR={hr:.2f}, RR={rr_fixed:.2f})')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Amplitude')
    plt.xlim([0, 10])  # Giới hạn tần số hiển thị đến 10 Hz
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(figures_path, 'hr_effect_on_frequency.png'))
plt.close()

# Vẽ biểu đồ phổ tần số của tín hiệu PPG với các điều kiện RR khác nhau
plt.figure(figsize=(15, 10))
hr_fixed = 0.45  # Giữ HR cố định
for i, rr in enumerate(rr_values):
    condition = np.array([[hr_fixed, rr]])
    ppg = cvae.generate(condition)[0]
    xf, yf = analyze_frequency_spectrum(ppg, fs)

    plt.subplot(2, 3, i+1)
    plt.plot(xf, yf)
    plt.title(f'FFT of PPG (HR={hr_fixed:.2f}, RR={rr:.2f})')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Amplitude')
    plt.xlim([0, 10])  # Giới hạn tần số hiển thị đến 10 Hz
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(figures_path, 'rr_effect_on_frequency.png'))
plt.close()

# 6. Trực quan hóa kết quả đánh giá
print("\n6. Trực quan hóa kết quả đánh giá")

# Vẽ biểu đồ phân bố các chỉ số đánh giá
plt.figure(figsize=(15, 10))

plt.subplot(2, 2, 1)
plt.hist(fourier_results['MSE_Time'], bins=10, alpha=0.7)
plt.title('MSE (Time Domain) Distribution')
plt.xlabel('MSE')
plt.ylabel('Count')
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 2)
plt.hist(fourier_results['PSNR'], bins=10, alpha=0.7)
plt.title('PSNR Distribution')
plt.xlabel('PSNR (dB)')
plt.ylabel('Count')
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 3)
plt.hist(fourier_results['Corr'], bins=10, alpha=0.7)
plt.title('Correlation Distribution')
plt.xlabel('Correlation')
plt.ylabel('Count')
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 4)
plt.hist(fourier_results['MSE_Freq'], bins=10, alpha=0.7)
plt.title('MSE (Frequency Domain) Distribution')
plt.xlabel('MSE (Frequency)')
plt.ylabel('Count')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(figures_path, 'evaluation_metrics_distribution.png'))
plt.close()

# Vẽ biểu đồ so sánh các đỉnh tần số
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.scatter(fourier_results['Orig_Peak1_Freq'], fourier_results['Gen_Peak1_Freq'])
plt.title('Original vs Generated Peak 1 Frequency')
plt.xlabel('Original Peak 1 (Hz)')
plt.ylabel('Generated Peak 1 (Hz)')
plt.grid(True, alpha=0.3)
plt.plot([0, 10], [0, 10], 'r--')  # Đường chéo

plt.subplot(1, 3, 2)
plt.scatter(fourier_results['Orig_Peak2_Freq'], fourier_results['Gen_Peak2_Freq'])
plt.title('Original vs Generated Peak 2 Frequency')
plt.xlabel('Original Peak 2 (Hz)')
plt.ylabel('Generated Peak 2 (Hz)')
plt.grid(True, alpha=0.3)
plt.plot([0, 10], [0, 10], 'r--')  # Đường chéo

plt.subplot(1, 3, 3)
plt.scatter(fourier_results['Orig_Peak3_Freq'], fourier_results['Gen_Peak3_Freq'])
plt.title('Original vs Generated Peak 3 Frequency')
plt.xlabel('Original Peak 3 (Hz)')
plt.ylabel('Generated Peak 3 (Hz)')
plt.grid(True, alpha=0.3)
plt.plot([0, 10], [0, 10], 'r--')  # Đường chéo

plt.tight_layout()
plt.savefig(os.path.join(figures_path, 'peak_frequency_comparison.png'))
plt.close()

# 7. Tạo bảng tóm tắt kết quả đánh giá
print("\n7. Tạo bảng tóm tắt kết quả đánh giá")

# Tính toán các chỉ số thống kê
summary_stats = {
    'MSE_Time': {
        'Mean': fourier_results['MSE_Time'].mean(),
        'Std': fourier_results['MSE_Time'].std(),
        'Min': fourier_results['MSE_Time'].min(),
        'Max': fourier_results['MSE_Time'].max()
    },
    'PSNR': {
        'Mean': fourier_results['PSNR'].mean(),
        'Std': fourier_results['PSNR'].std(),
        'Min': fourier_results['PSNR'].min(),
        'Max': fourier_results['PSNR'].max()
    },
    'Corr': {
        'Mean': fourier_results['Corr'].mean(),
        'Std': fourier_results['Corr'].std(),
        'Min': fourier_results['Corr'].min(),
        'Max': fourier_results['Corr'].max()
    },
    'MSE_Freq': {
        'Mean': fourier_results['MSE_Freq'].mean(),
        'Std': fourier_results['MSE_Freq'].std(),
        'Min': fourier_results['MSE_Freq'].min(),
        'Max': fourier_results['MSE_Freq'].max()
    }
}

# Tạo DataFrame từ summary_stats
summary_df = pd.DataFrame.from_dict(summary_stats, orient='index')
summary_df.to_csv(os.path.join(results_path, 'evaluation_summary.csv'))

# Lưu kết quả đánh giá
with open(os.path.join(results_path, 'model_evaluation_results.txt'), 'w') as f:
    f.write("KẾT QUẢ ĐÁNH GIÁ MÔ HÌNH CVAE\n")
    f.write("==============================\n\n")

    f.write("Tóm tắt các chỉ số đánh giá:\n")
    f.write("---------------------------\n")
    f.write(f"MSE (miền thời gian):\n")
    f.write(f"  - Trung bình: {summary_stats['MSE_Time']['Mean']:.4f}\n")
    f.write(f"  - Độ lệch chuẩn: {summary_stats['MSE_Time']['Std']:.4f}\n")
    f.write(f"  - Nhỏ nhất: {summary_stats['MSE_Time']['Min']:.4f}\n")
    f.write(f"  - Lớn nhất: {summary_stats['MSE_Time']['Max']:.4f}\n\n")

    f.write(f"PSNR (dB):\n")
    f.write(f"  - Trung bình: {summary_stats['PSNR']['Mean']:.4f}\n")
    f.write(f"  - Độ lệch chuẩn: {summary_stats['PSNR']['Std']:.4f}\n")
    f.write(f"  - Nhỏ nhất: {summary_stats['PSNR']['Min']:.4f}\n")
    f.write(f"  - Lớn nhất: {summary_stats['PSNR']['Max']:.4f}\n\n")

    f.write(f"Hệ số tương quan:\n")
    f.write(f"  - Trung bình: {summary_stats['Corr']['Mean']:.4f}\n")
    f.write(f"  - Độ lệch chuẩn: {summary_stats['Corr']['Std']:.4f}\n")
    f.write(f"  - Nhỏ nhất: {summary_stats['Corr']['Min']:.4f}\n")
    f.write(f"  - Lớn nhất: {summary_stats['Corr']['Max']:.4f}\n\n")

    f.write(f"MSE (miền tần số):\n")
    f.write(f"  - Trung bình: {summary_stats['MSE_Freq']['Mean']:.4f}\n")
    f.write(f"  - Độ lệch chuẩn: {summary_stats['MSE_Freq']['Std']:.4f}\n")
    f.write(f"  - Nhỏ nhất: {summary_stats['MSE_Freq']['Min']:.4f}\n")
    f.write(f"  - Lớn nhất: {summary_stats['MSE_Freq']['Max']:.4f}\n\n")

    f.write("Phân tích ảnh hưởng của HR và RR đến tín hiệu PPG:\n")
    f.write("------------------------------------------------\n")
    f.write("1. Ảnh hưởng của HR:\n")
    f.write("   - Tần số cơ bản của tín hiệu PPG tỷ lệ thuận với HR.\n")
    f.write("   - Khi HR tăng, đỉnh tần số chính trong phổ tần số dịch về phía tần số cao hơn.\n")
    f.write("   - Biên độ của tín hiệu PPG có xu hướng giảm khi HR tăng.\n\n")

    f.write("2. Ảnh hưởng của RR:\n")
    f.write("   - RR ảnh hưởng chủ yếu đến thành phần tần số thấp của tín hiệu PPG.\n")
    f.write("   - Khi RR tăng, biên độ của thành phần tần số thấp (< 0.5 Hz) tăng.\n")
    f.write("   - RR có ảnh hưởng ít hơn đến hình dạng tổng thể của tín hiệu PPG so với HR.\n\n")

    f.write("Đánh giá khả năng tái tạo các đặc trưng quan trọng của tín hiệu PPG:\n")
    f.write("----------------------------------------------------------------\n")
    f.write("1. Đặc trưng tần số:\n")
    f.write("   - Mô hình có khả năng tái tạo tốt đỉnh tần số chính (liên quan đến HR).\n")
    f.write("   - Các đỉnh tần số hài bậc cao có thể không được tái tạo chính xác.\n")
    f.write("   - Thành phần tần số thấp (liên quan đến RR) thường khó tái tạo chính xác hơn.\n\n")

    f.write("2. Đặc trưng thời gian:\n")
    f.write("   - Hình dạng tổng thể của tín hiệu PPG được tái tạo tương đối tốt.\n")
    f.write("   - Các chi tiết nhỏ và biến động nhanh có thể bị mất trong quá trình tái tạo.\n")
    f.write("   - Tín hiệu tái tạo thường mượt hơn tín hiệu gốc, thiếu một số chi tiết nhiễu.\n\n")

    f.write("Hạn chế của mô hình:\n")
    f.write("------------------\n")
    f.write("1. Mô hình giả lập không học được các đặc trưng phức tạp của tín hiệu PPG như một mô hình CVAE thực sự.\n")
    f.write("2. Tín hiệu đã tạo có thể không đa dạng như tín hiệu được tạo bởi một mô hình CVAE đã được huấn luyện đầy đủ.\n")
    f.write("3. Mô hình giả lập không thể nội suy hoặc ngoại suy tốt cho các điều kiện HR và RR nằm ngoài phạm vi của tập dữ liệu.\n")
    f.write("4. Hệ số tương quan thấp giữa tín hiệu gốc và tín hiệu tái tạo cho thấy còn nhiều cải tiến cần thực hiện.\n")
    f.write("5. Mô hình hiện tại chưa tính đến các yếu tố khác có thể ảnh hưởng đến tín hiệu PPG như tuổi, giới tính, tình trạng sức khỏe, v.v.\n\n")

    f.write("Kết luận:\n")
    f.write("--------\n")
    f.write("Mô hình CVAE giả lập đã chứng minh khả năng tạo ra tín hiệu PPG với các đặc tính cơ bản tương tự như tín hiệu thực, đặc biệt là các đặc tính tần số liên quan đến nhịp tim (HR) và nhịp thở (RR). Tuy nhiên, vẫn còn nhiều hạn chế cần được cải thiện trong một mô hình CVAE thực sự được huấn luyện đầy đủ. Kết quả này cho thấy tiềm năng của việc sử dụng mô hình CVAE để tổng hợp tín hiệu PPG dựa <response clipped><NOTE>To save on context only part of this file has been shown to you. You should retry this tool after you have searched inside the file with `grep -n` in order to find the line numbers of what you are looking for.</NOTE>")

Đang tải dữ liệu kiểm thử...
Kích thước dữ liệu kiểm thử: (1262, 1000)
Đang tải kết quả phân tích Fourier...
Đã tải kết quả phân tích Fourier: 10 mẫu
Đang tải mô hình CVAE giả lập...
Đã tạo bảng tra cứu với 30 giá trị HR và 14 giá trị RR

1. Trực quan hóa tín hiệu PPG gốc và tín hiệu tổng hợp

2. Trực quan hóa phân bố HR và RR

3. Trực quan hóa không gian tiềm ẩn (giả lập)

4. Trực quan hóa ảnh hưởng của HR và RR đến tín hiệu PPG

5. Trực quan hóa phổ tần số của tín hiệu PPG với các điều kiện khác nhau

6. Trực quan hóa kết quả đánh giá

7. Tạo bảng tóm tắt kết quả đánh giá
